In [1]:
import pandas
import gzip
import time
import argparse
import pickle
from Class import *
import function
import tqdm

In [2]:
__version__ = "V0.0(Editor) 2023-10-24"

In [3]:
# 前置参数-debug
COUNTSCUTOFF = 2
ABSOLUTEPATH = False
FILEPATH = "F:/OneDrive/Master/Project/trans/data/"
RAWDATAPATH = "raw_data/"
INPUTTOTAL = "10001_total.pkl"
INPUTREFANNOTATION = "gencode.v29.primary_assembly.annotation_UCSC_names.gtf.gz"
OUTPUTTOTAL = "10002_total.pkl"

In [4]:
# 补全路径
if ABSOLUTEPATH == False:
    RAWDATAPATH = "{}{}".format(FILEPATH, RAWDATAPATH)
    INPUTTOTAL = "{}{}".format(FILEPATH, INPUTTOTAL)
    INPUTREFANNOTATION = "{}annotation/{}".format(RAWDATAPATH, INPUTREFANNOTATION)
    OUTPUTTOTAL = "{}{}".format(FILEPATH, OUTPUTTOTAL)

In [5]:
# 引用变量
EXONRANGE = 5
RANGETSS = 50
RANGETES = 150
GENERANGETSS = 50
GENERANGETES = 150

In [6]:
# 一级函数
def findId(index, chr, location, rangeMax, orientation, allowType, total=None, transcriptGeneMap=None):
    '''
    input:
        total, Total Object
        index, dict, {<chr>: {<location>: {<location>: <id>, ...}, ...}, ...}
        chr, str
        location, int
        rangeMax, int
        orientation, str, 当为+时, location递增; 当为-时, location递减
        allowType, str, "gene" or "transcript" or "exon"
        transcriptGeneMap, dict, {<transcriptId>: <geneId>, ...}
    change:
        寻找index中, 相距location为rangeMax的id(不包含location)
    '''
    total = total
    index = index
    chr = chr
    location = location
    rangeMax = rangeMax
    orientation = orientation
    allowType = allowType
    transcriptGeneMap = transcriptGeneMap

    temp = []  # [(<id>, <mean expression in all cellLine>)]

    if orientation == '-':
        rangeList = [i for i in range(location-1, location-rangeMax-1, -1)]
    else:
        rangeList = [i for i in range(location+1, location+rangeMax+1, 1)]

    for i in rangeList:
        # 寻找是否在指定范围内存在相应的location
        if index[chr].get(i, None) is None:
            # 如果不存在, 就直接判断下一个位置
            continue
        else:
            # 寻找到了范围内的location
            newLocation = i
            if allowType == "exon":
                # index[chr][newLocation].values()中的值为str, 而不是list
                for id in index[chr][newLocation].values():
                    if total is not None:
                        expression = numpy.mean([total.exonDict[id].cellLineExpression.get(cellLine, 0) for cellLine in total.celllineInfo.keys()])
                    else:
                        expression = None
                    temp.append((id, expression))
            elif allowType == "gene":
                # index[chr][newLocation].values()中的值为str, 而不是list
                for id in index[chr][newLocation].values():
                    if total is not None:
                        expression = numpy.mean([total.geneDict[id].cellLineExpression.get(cellLine, 0) for cellLine in total.celllineInfo.keys()])
                    else:
                        expression = None
                    temp.append((id, expression))
            else:
                # index[chr][newLocation].values()中的值为list, 而不是str
                if transcriptGeneMap is None:
                    raise ValueError("findId(): need transcriptGeneMap")
                for idList in index[chr][newLocation].values():
                    for id in idList:
                        if total is not None:
                            geneId = transcriptGeneMap.get(id, None)
                            if geneId is None:
                                expression = None
                            else:
                                expression = numpy.mean([total.geneDict[geneId].transcriptDict[id].cellLineExpression.get(cellLine, 0) for cellLine in total.celllineInfo.keys()])
                        else:
                            expression = None
                        temp.append(((geneId, id, expression), (id, expression))[total is None])
            temp = temp + findId(total=total, index=index, chr=chr, location=newLocation, rangeMax=rangeMax, orientation=orientation, allowType=allowType, transcriptGeneMap=transcriptGeneMap)

    # 去重
    temp = list(set(temp))

    return temp
    

In [7]:
# 一级函数
def loadRefAnnotation(filename, allowType):
    '''
    input:
        filename, str, 参考基因组注释文件位置.gz
        allowType, str, "transcript" or "gene" or "exon"
    change:
        读取参考基因组注释数据中allowType的数据
        Warning: 在读取name时, 如果没有name则会返回"None"
    return:
        dict, {"positionRef": {"TSS": {<chr>: {<TSS>: {<TES>: [<transcriptId>, ...],
                                                               ...},
                                                       ...},
                                               ...},
                                       ...},
                               "TES": {<chr>: {<TES>: {<TSS>: [<transcriptId>, ...],
                                                               ...},
                                                       ...},
                                               ...}
                                       ...},
                "dataRef": {<id>: {<key>: <value>,
                                   ...},
                            ...}
    '''
    # 读取参考基因组注释数据
    filename = filename
    allowType = [allowType]

    positionRef = {"TSS": {}, "TES": {}}  # {"TSS": {<chr>: {<TSS>: {<TES>: [<id>, ...], ...}, ...}, ...}, "TES": {<chr>: {<TES>: {<TSS>: [<id>, ...], ...}, ...}, ...}}
    dataRef = {}

    # 读取数据
    with gzip.open(filename, 'rt') as file:
        if allowType == ["transcript"]:
            for line in file:
                line = function.pickRefAnnotation(info=line, allowType=allowType)
                if line is None:
                    continue
                # 提取数据
                [geneId, geneVersion] = function.strSplit(line["gene_id"])
                geneName = line.get("gene_name", "None")
                geneBiotype = line["gene_type"]
                [transcriptId, transcriptVersion] = function.strSplit(line["transcript_id"])  # 去掉版本号
                transcriptName = line.get("transcript_name", "None")
                transcriptBiotype = line["transcript_type"]
                strand = line["strand"]
                chr = line["chr"]
                chr = (chr, 'M')[chr == "MT"]
                start = int(line["start"])
                end = int(line["end"])
                # 判断转录起点与转录终点
                if strand == '+':
                    TSS, TES = start, end
                else:
                    TSS, TES = end, start
                # 在dataRef中添加信息
                if transcriptId in dataRef.keys():
                    print("[Warning]loadRefAnnotation(): transcriptId {} have been existed".format(transcriptId))
                    print("existed: {}".format(dataRef[transcriptId]))
                    print("new: {}".format({"geneId": geneId,"geneVersion": geneVersion,"geneName": geneName,"geneBiotype": geneBiotype,"transcriptId": transcriptId,"transcriptVersion": transcriptVersion,"transcriptName": transcriptName,"transcriptBiotype": transcriptBiotype,"strand": strand,"chr": chr,"start": start,"end": end,"TSS": TSS,"TES": TES}))
                dataRef[transcriptId] = {"geneId": geneId,
                                         "geneVersion": geneVersion,
                                         "geneName": geneName,
                                         "geneBiotype": geneBiotype,
                                         "transcriptId": transcriptId,
                                         "transcriptVersion": transcriptVersion,
                                         "transcriptName": transcriptName,
                                         "transcriptBiotype": transcriptBiotype,
                                         "strand": strand,
                                         "chr": chr,
                                         "start": start,
                                         "end": end,
                                         "TSS": TSS,
                                         "TES": TES}
        elif allowType == ["gene"]:
            for line in file:
                line = function.pickRefAnnotation(info=line, allowType=allowType)
                if line is None:
                    continue
                # 提取数据
                [geneId, geneVersion] = function.strSplit(line["gene_id"])
                geneName = line.get("gene_name", "None")
                geneBiotype = line["gene_type"]
                strand = line["strand"]
                chr = line["chr"]
                chr = (chr, 'M')[chr == "MT"]
                start = int(line["start"])
                end = int(line["end"])
                # 判断转录起点与转录终点
                if strand == '+':
                    TSS, TES = start, end
                else:
                    TSS, TES = end, start
                # 在dataRef中添加信息
                if geneId in dataRef.keys():
                    print("[Warning]loadRefAnnotation(): geneId {} have been existed".format(geneId))
                dataRef[geneId] = {"geneId": geneId,
                                    "geneName": geneName,
                                    "geneVersion": geneVersion,
                                    "geneBiotype": geneBiotype,
                                    "strand": strand,
                                    "chr": chr,
                                    "start": start,
                                    "end": end,
                                    "TSS": TSS,
                                    "TES": TES}
        elif allowType == ["exon"]:
            for line in file:
                line = function.pickRefAnnotation(info=line, allowType=allowType)
                if line is None:
                    continue
                # 提取数据
                [geneId, geneVersion] = function.strSplit(line["gene_id"])
                [transcriptId, transcriptVersion] = function.strSplit(line["transcript_id"])
                [exonId, exonVersion] = function.strSplit(line["exon_id"])
                strand = line["strand"]
                chr = line["chr"]
                chr = (chr, 'M')[chr == "MT"]
                start = int(line["start"])
                end = int(line["end"])
                # 判断转录起点与转录终点
                if strand == '+':
                    TSS, TES = start, end
                else:
                    TSS, TES = end, start
                # 在dataRef中添加信息
                if exonId in dataRef.keys():
                    dataRef[exonId]["geneTranscriptIdSet"].add((geneId, transcriptId))
                else:
                    dataRef[exonId] = {"geneId": geneId,
                                        "transcriptId": transcriptId,
                                        "exonId": exonId,
                                        "exonVersion": exonVersion,
                                        "strand": strand,
                                        "chr": chr,
                                        "start": start,
                                        "end": end,
                                        "TSS": TSS,
                                        "TES": TES,
                                        "geneTranscriptIdSet": {(geneId, transcriptId)}}
        else:
            raise ValueError("the allowType should be 'transcript' or 'gene', but it is {}".format(allowType))

    # 整理index
    # 在positionRef中添加信息
    if allowType == ["transcript"]:
        for id, transcriptDict in dataRef.items():
            chr = transcriptDict["chr"]
            TSS = transcriptDict["TSS"]
            TES = transcriptDict["TES"]
            if chr not in positionRef["TSS"].keys():
                positionRef["TSS"][chr] = {TSS: {TES: [id]}}
            else:
                if TSS not in positionRef["TSS"][chr].keys():
                    positionRef["TSS"][chr][TSS] = {TES: [id]}
                else:
                    if TES not in positionRef["TSS"][chr][TSS].keys():
                        positionRef["TSS"][chr][TSS][TES] = [id]
                    else:
                        positionRef["TSS"][chr][TSS][TES].append(id)
            if chr not in positionRef["TES"].keys():
                positionRef["TES"][chr] = {TES: {TSS: [id]}}
            else:
                if TES not in positionRef["TES"][chr].keys():
                    positionRef["TES"][chr][TES] = {TSS: [id]}
                else:
                    if TSS not in positionRef["TES"][chr][TES].keys():
                        positionRef["TES"][chr][TES][TSS] = [id]
                    else:
                        positionRef["TES"][chr][TES][TSS].append(id)
    elif allowType == ["gene"] or allowType == ["exon"]:
        for id, dataDict in dataRef.items():
            chr = dataDict["chr"]
            TSS = dataDict["TSS"]
            TES = dataDict["TES"]
            if chr not in positionRef["TSS"].keys():
                positionRef["TSS"][chr] = {TSS: {TES: id}}
            else:
                if TSS not in positionRef["TSS"][chr].keys():
                    positionRef["TSS"][chr][TSS] = {TES: id}
                else:
                    if TES not in positionRef["TSS"][chr][TSS].keys():
                        positionRef["TSS"][chr][TSS][TES] = id
                    else:
                        pass
                        #print("[Warning]{id}-->{e}, but {e} is existed".format(id=id, e=positionRef["TSS"][chr][TSS][TES]))
            if chr not in positionRef["TES"].keys():
                positionRef["TES"][chr] = {TES: {TSS: id}}
            else:
                if TES not in positionRef["TES"][chr].keys():
                    positionRef["TES"][chr][TES] = {TSS: id}
                else:
                    if TSS not in positionRef["TES"][chr][TES].keys():
                        positionRef["TES"][chr][TES][TSS] = id
                    else:
                        pass
                        #print("[Warning]{id}-->{e}, but {e} is existed".format(id=id, e=positionRef["TSS"][chr][TSS][TES]))
    else:
        raise ValueError("the allowType should be 'transcript' or 'gene', but it is {}".format(allowType))
    
    return {"positionRef": positionRef, "dataRef": dataRef}
        

In [8]:
# load data
with open(INPUTTOTAL, "rb") as file:
    total = pickle.load(file)

In [9]:
# 读取参考基因组注释中gene数据
temp = loadRefAnnotation(filename=INPUTREFANNOTATION, allowType="gene")
# 获取所有transcript信息
geneRef = temp["dataRef"]
# 获取index
geneIndex = temp["positionRef"]

# 读取参考基因组注释中transcript数据
temp = loadRefAnnotation(filename=INPUTREFANNOTATION, allowType="transcript")
# 获取所有transcript信息
transcriptRef = temp["dataRef"]
# 获取index
transcriptIndex = temp["positionRef"]

# 读取参考基因组注释中exon的数据, 并建立index
temp = loadRefAnnotation(filename=INPUTREFANNOTATION, allowType="exon")
exonRef = temp["dataRef"]
exonIndex = temp["positionRef"]

In [10]:
# 补充transcript中的exon信息
keys = set(transcriptRef)
for exonId, exonDict in exonRef.items():
    for (geneId, transcriptId) in exonDict["geneTranscriptIdSet"]:
        exonSet = transcriptRef[transcriptId].setdefault("exonSet", set())
        exonSet.add(exonId)

In [11]:
# 更新total中的exonDict及exonIndex
total.refresh()
total.reIndex()

In [12]:
# 声明变量
exonCorrected = {"mapped": {}, "TSS-TES": {}, "TSS": {}, "TES": {}, "integrate TSS": {}, "integrate TES": {}}  # {"": {<existed exon id>: <corrected exon id>}, ...}
readyCorList = []

In [13]:
# 根据参考基因组中exon数据对exon的TSS及TES进行校正
# 遍历total.exonDict中所有exon, 记录这些exon
for exonId, exonObject in total.exonDict.items():
    readyCorList.append(exonId)
# 统计校正前exon及transcript的数量
infoDict = {}
infoDict["total exon num before exon correct"] = len(total.exonDict)
num = 0
for exonId, exonObject in total.exonDict.items():
    if exonObject.status == "NOVEL":
        num += 1
infoDict["novel exon num before exon correct"] = num
num = 0
for geneId, geneObject in total.geneDict.items():
    for transcriptId in geneObject.transcriptDict.keys():
        num += 1
infoDict["total transcript num before exon correct"] = num
num = 0
for geneId, geneObject in total.geneDict.items():
    for transcriptObject in geneObject.transcriptDict.values():
        if transcriptObject.status == "NOVEL":
            num +=1
infoDict["novel transcript num before exon correct"] = num

# 根据参考基因组中exon数据对exon的TSS及TES进行校正
for exonId in tqdm.tqdm(readyCorList, desc="Exon", leave=True):
    exonObject = total.exonDict[exonId]
    chr = exonObject.chr
    TSS = exonObject.TSS
    TES = exonObject.TES

    # 声明list类型变量exonList
    exonList = []  # 存储可进行映射的exon [(<exonId>, <dif>, <startMarker>, <endMarker>), ...]

    # 准备校正exon的TSS及TES
    # 获取[TSS-5, TES+5]范围的所有exon
    temp = [exonIndex["TSS"][chr].get(i, None) for i in range(TSS-EXONRANGE, TSS+EXONRANGE+1)]
    # 去除空值
    while None in temp:
        temp.remove(None)
    while {} in temp:
        temp.remove({})
        pass
    for d in temp:
        for tempTES, tempExonId in d.items():
            if abs(tempTES-TES) <= EXONRANGE:
                # 从中筛选出[TES-5, TES+5]范围的所有exon, 认为这些exon为可双端映射的exon
                num = abs(TSS-exonRef[tempExonId]["TSS"]) + abs(TES-tempTES)
                exonList.append((tempExonId, num, True, True))
            else:
                # 仅TSS可映射到tempExon
                num = abs(TSS-exonRef[tempExonId]["TSS"])
                exonList.append((tempExonId, num, True, False))

    # 获取[TES-5, TES+5]范围的所有exon
    temp = [exonIndex["TES"][chr].get(i, None) for i in range(TES-EXONRANGE, TES+EXONRANGE+1)]
    # 去除空值
    while None in temp:
        temp.remove(None)
    while {} in temp:
        temp.remove({})
    for d in temp:
        for tempTSS, tempExonId in d.items():
            # 从中筛选出(-oo, TSS-5)U(TSS+5, +oo)的所有exon, 认为这些exon为仅可映射到TES的exon
            if abs(tempTSS-TSS) > EXONRANGE:
                # 仅end可映射到tempExon
                num = abs(TES-exonRef[tempExonId]["TES"])
                exonList.append((tempExonId, num, False, True))

    # 不分析无法映射的exon
    if len(exonList) == 0:
        continue

    # 校正过程
    mode = False  # 是否存在最佳映射
    for (newExonId, num, startMarker, endMarker) in exonList:
        if startMarker == endMarker == True:
            mode = True
            break
    # 开始映射
    if mode is True:
        # 若exon可双端映射到一个exon
        # 获取相差碱基数最少的exon
        temp = []
        for (newExonId, num, TSSMarker, TESMarker) in exonList:
            if TSSMarker == TESMarker == True:
                temp.append((newExonId, num))
        temp = sorted(temp, key=lambda x: x[1])
        (newExonId, num) = temp[0]
        '''if num == 0:
            # 表明不必进行start, end的修改
            continue'''
        # 判断exonId是否相同
        if exonId == newExonId:
            # exonId相同
            # 修改status, exonVersion, start, TSS, end, TES
            total.exonDict[exonId].status = "KNOWN"
            total.exonDict[exonId].exonVersion = exonRef[newExonId]["exonVersion"]
            total.exonDict[exonId].start = exonRef[newExonId]["start"]
            total.exonDict[exonId].end = exonRef[newExonId]["end"]
            total.exonDict[exonId].TSS = exonRef[newExonId]["TSS"]
            total.exonDict[exonId].TES = exonRef[newExonId]["TES"]
        else:
            # exonId不同
            # 查询参考exonId是否已记录
            if newExonId in total.exonDict.keys():
                # 参考exonId已记录, 合并两exon
                total._exonMerge(exonId1=newExonId, exonId2=exonId)
            else:
                # 参考exonId未记录
                # exonDict中新增参考exon对象
                total.exonDict[newExonId] = Exon(status="KNOWN",
                                                 exonId=exonRef[newExonId]["exonId"],
                                                 exonVersion=exonRef[newExonId]["exonVersion"],
                                                 chr=exonRef[newExonId]["chr"],
                                                 strand=exonRef[newExonId]["strand"],
                                                 start=exonRef[newExonId]["start"],
                                                 end=exonRef[newExonId]["end"],
                                                 TSS=exonRef[newExonId]["TSS"],
                                                 TES=exonRef[newExonId]["TES"])
                # 合并原exon与参考exon
                total._exonMerge(exonId1=newExonId, exonId2=exonId)
            # 将映射关系{<exonId>: <newExonId>}保存到exonCorrected["mapped"]中
            exonCorrected["mapped"][exonId] = newExonId
    else:
        # 若exon不可双端映射到一个exon
        # 如果status为KNOWN就不再进行映射, 跳过该exon
        if total.exonDict[exonId].status == "KNOWN":
            continue
        # 修改exon的status为CORRECTED
        total.exonDict[exonId].status = "CORRECTED"
        # 判断exon能否双端映射到两个exon
        marker=False  # 标记该exon能否映射到两个exon, 1-可根据start映射 2-可根据end映射 3-可分别根据start,end映射
        for (newExonId, num, startMarker, endMarker) in exonList:
            if startMarker is True:
                marker = 3 if marker==2 else 1
            else:
                marker = 3 if marker==1 else 2
            if marker == 3:
                break
        if marker == 3:
            # 能映射到两个exon
            # 获取TSS端相差碱基数最少的exon
            temp = [(newExonId, num, startMarker, endMarker) for (newExonId, num, startMarker, endMarker) in exonList if startMarker is True]
            temp = sorted(temp, key=lambda x: x[1])
            (newExonId, num, startMarker, endMarker) = temp[0]
            # 在exon的status中添加-TSS-TES后缀
            total.exonDict[exonId].status = total.exonDict[exonId].status + "-TSS-TES"
            if num != 0:
                # 表明需要进行TSS的修改
                # 修改exon的TSS的位置
                if total.exonDict[exonId].strand == '+':
                    total.exonDict[exonId].start = exonRef[newExonId]["start"]
                    total.exonDict[exonId].TSS = exonRef[newExonId]["TSS"]
                else:
                    total.exonDict[exonId].end = exonRef[newExonId]["end"]
                    total.exonDict[exonId].TSS = exonRef[newExonId]["TSS"]
                # 将映射关系{<exonId>: <newExonId>}保存到exonCorrected["TSS-TES"]中
                exonCorrected["TSS-TES"][exonId] = {"TSS": newExonId, "TES": None}
            # 获取TES端相差碱基数最少的exon
            temp = [(newExonId, num, startMarker, endMarker) for (newExonId, num, startMarker, endMarker) in exonList if endMarker is True]
            temp = sorted(temp, key=lambda x: x[1])
            (newExonId, num, startMarker, endMarker) = temp[0]
            if num != 0:
                # 表明需要进行TES的修改
                # 修改exon的TES的位置
                if total.exonDict[exonId].strand == '+':
                    total.exonDict[exonId].end = exonRef[newExonId]["end"]
                    total.exonDict[exonId].TES = exonRef[newExonId]["TES"]
                else:
                    total.exonDict[exonId].start = exonRef[newExonId]["start"]
                    total.exonDict[exonId].TES = exonRef[newExonId]["TES"]
                # 将映射关系{<exonId>: <newExonId>}保存到exonCorrected["TSS-TES"]中
                if exonId not in exonCorrected["TSS-TES"].keys():
                    exonCorrected["TSS-TES"][exonId] = {"TSS": None, "TES": newExonId}
                else:
                    exonCorrected["TSS-TES"][exonId]["TES"] = newExonId
        else:
            # 仅能单端映射到一个exon
            # 筛选出相差碱基数最少的exon进行映射
            exonList = sorted(exonList, key=lambda x: x[1])
            tempNum = exonList[0][1]
            exonList = [(newExonId, num, startMarker, endMarker) for (newExonId, num, startMarker, endMarker) in exonList if num == tempNum]
            (newExonId, num, startMarker, endMarker) = exonList[0]
            # 判断哪一端可进行映射
            if startMarker is True:
                # TSS端可映射
                # 在exon的status中添加-TSS后缀
                total.exonDict[exonId].status = total.exonDict[exonId].status + "-TSS"
                # 将映射关系{<exonId>: <newExonId>}保存到exonCorrected["TSS"]中
                exonCorrected["TSS"][exonId] = newExonId
                if num == 0:
                    # 表明不必进行TSS的修改
                    continue
                # 修改exon的TSS的位置
                if total.exonDict[exonId].strand == '+':
                    total.exonDict[exonId].start = exonRef[newExonId]["start"]
                    total.exonDict[exonId].TSS = exonRef[newExonId]["TSS"]
                else:
                    total.exonDict[exonId].end = exonRef[newExonId]["end"]
                    total.exonDict[exonId].TSS = exonRef[newExonId]["TSS"]
            else:
                # TES端可映射
                # 在exon的status中添加-TES后缀
                total.exonDict[exonId].status = total.exonDict[exonId].status + "-TES"
                # 将映射关系{<exonId>: <newExonId>}保存到exonCorrected["TES"]中
                exonCorrected["TES"][exonId] = newExonId
                if num == 0:
                    # 表明不必进行TES的修改
                    continue
                # 修改exon的TES的位置
                if total.exonDict[exonId].strand == '+':
                    total.exonDict[exonId].end = exonRef[newExonId]["end"]
                    total.exonDict[exonId].TES = exonRef[newExonId]["TES"]
                else:
                    total.exonDict[exonId].start = exonRef[newExonId]["start"]
                    total.exonDict[exonId].TES = exonRef[newExonId]["TES"]

Exon: 100%|██████████| 323363/323363 [01:17<00:00, 4189.82it/s] 


In [14]:
# 更新total中的exonDict及exonIndex
total.refresh()
total.reIndex()
# 计算exon, transcript, gene的相对表达量
total.computeRelativeExpression()
total.computeCellLineExpression()

In [15]:
# 统计校正后exon及transcript的数量
infoDict["exon mapped"] = len(exonCorrected["mapped"])
infoDict["exon TSS-TES"] = len(exonCorrected["TSS-TES"])
infoDict["exon TSS"] = len(exonCorrected["TSS"])
infoDict["exon TES"] = len(exonCorrected["TES"])
infoDict["total exon num after exon correct"] = len(total.exonDict)
num=0
for exonId, exonObject in total.exonDict.items():
    if exonObject.status == "NOVEL":
        num+=1
infoDict["novel exon num after exon correct"] = num
num = 0
for geneId, geneObject in total.geneDict.items():
    for transcriptId in geneObject.transcriptDict.keys():
        num += 1
infoDict["total transcript num after exon correct"] = num
num = 0
for geneId, geneObject in total.geneDict.items():
    for transcriptObject in geneObject.transcriptDict.values():
        if transcriptObject.status == "NOVEL":
            num +=1
infoDict["novel transcript num after exon correct"] = num

In [16]:
# 准备记录合并记录
infoDict["integrate TSS"] = 0
infoDict["integrate TES"] = 0

In [17]:
# 合并相邻的exon, 即合并相差EXONRANGE范围内的NOVEL及未校正端的exon
if True:
    # 建立index
    indexTSS = total.exonIndexBuild(siteType="TSS")  # {<chr>: {<TSS>: {<TES>: <exonId>, ...}, ...}, ...}
    indexTES = total.exonIndexBuild(siteType="TES")  # {<chr>: {<TES>: {<TSS>: <exonId>, ...}, ...}, ...}

    # 处理TSS
    # 声明set变量exonSet
    exonSet = set()  # {<exonId>, ...}
    # 筛选符合校正条件的exon
    for exonId, exonObject in total.exonDict.items():
        if exonObject.status == "KNOWN":
            continue
        elif "-TSS" in exonObject.status:
            continue
        else:
            exonSet.add(exonId)
    # 只要exonSet不为空集, 就随机提取exonSet中的一个exonId, 寻找其TSS上下游各EXONRANGE范围内的exon
    while len(exonSet) != 0:
        exonId = exonSet.pop()
        exonObject = total.exonDict[exonId]
        chr = exonObject.chr
        TSS = exonObject.TSS
        # 声明list变量temp, [(<exonId>, <expression>), ...]
        temp = []  # [(<exonId>, <expression>), ...]
        # 寻找所有与该exon的start相同的exon
        for exonId in indexTSS[chr][TSS].values():
            temp = temp + [(exonId, numpy.mean([total.exonDict[exonId].cellLineExpression.get(cellLine, 0) for cellLine in total.celllineInfo.keys()]))]
        # 递减寻找EXONRANGE范围内的exon
        temp = temp + findId(total=total, index=indexTSS, chr=chr, location=TSS, rangeMax=EXONRANGE, orientation='-', allowType="exon")
        # 递增寻找EXONRANGE范围内的exon
        temp = temp + findId(total=total, index=indexTSS, chr=chr, location=TSS, rangeMax=EXONRANGE, orientation='+', allowType="exon")
        # 去重
        temp = list(set(temp))
        # 判断是否寻找到了其他的exon
        if len(temp)==1:
            # 未寻找到其他的exon, 认为该exon的start无法修改, 跳过该exon
            exonSet.discard(temp[0][0])
            continue
        else:
            # 已寻找到其他的exon
            pass
        # 判断在temp中是否存在已校正端
        corSet = set()  # {(exonId, TSS), ...}
        nonCorSet = set()  # {(exonId, TSS), ...}
        # 分离其中的未校正端与已校正端
        for (exonId, expression) in temp:
            status = total.exonDict[exonId].status
            TSS = total.exonDict[exonId].TSS
            if status=="KNOWN" or "-TSS" in status:
                # TSS端可信
                corSet.add((exonId, TSS))
            else:
                # TSS端不可信
                nonCorSet.add((exonId, TSS))
        if len(corSet) != 0:
            # 存在已校正端
            # 对每一个未校正端选择其距离最近的已校正端作为参考位点进行校正
            for (exonId, TSS) in nonCorSet:
                # 寻找最近的已校正端
                diffTSS = [(exonIdTemp, tempTSS, abs(TSS-tempTSS)) for (exonIdTemp, tempTSS) in corSet]
                diffTSS = sorted(diffTSS, key=lambda x: x[2])
                targetExonId = diffTSS[0][0]
                targetTSS = diffTSS[0][1]
                # 修改TSS, start/end位置
                total.exonDict[exonId].TSS = targetTSS
                if total.exonDict[exonId].strand == '+':
                    total.exonDict[exonId].start = targetTSS
                else:
                    total.exonDict[exonId].end = targetTSS
                # 改变status
                total.exonDict[exonId].status = total.exonDict[exonId].status + "-TSS"
                # 删除exonSet中该exonId
                exonSet.discard(exonId)
                # 记录这一次校正
                exonCorrected["integrate TSS"][exonId] = targetExonId
            # 结束此次循环
            continue
        else:
            # 不存在已校正端, 进行下一步
            pass
        # 以expression最大的exon为参照, 修改其他exon的TSS
        exonIdTarget = None
        expressionMax = 0
        # 寻找expression最大的exon
        for (exonId, expression) in temp:
            if expression >= expressionMax:
                expressionMax = expression
                exonIdTarget = exonId
            else:
                continue
        # 修改其他的exon
        for (exonId, expression) in temp:
            # 修改其他exon的TSS, start/end
            total.exonDict[exonId].TSS = total.exonDict[exonIdTarget].TSS
            if total.exonDict[exonId].strand == '+':
                total.exonDict[exonId].start = total.exonDict[exonIdTarget].start
            else:
                total.exonDict[exonId].end = total.exonDict[exonIdTarget].end
            # 修改status
            # 不应该修改status, 因为无已校正TSS作为参考, 所以TSS即使进行了合并也并不是可靠的
            #total.exonDict[exonId].status = total.exonDict[exonId].status + "-TSS"
            # 删除exonSet中的这些exon
            exonSet.discard(exonId)
            # 记录这一次校正
            exonCorrected["integrate TSS"][exonId] = exonIdTarget

    # 处理TES
    # 声明set变量exonSet
    exonSet = set()  # {<exonId>, ...}
    # 筛选符合校正条件的exon
    for exonId, exonObject in total.exonDict.items():
        if exonObject.status == "KNOWN":
            continue
        elif "-TES" in exonObject.status:
            continue
        else:
            exonSet.add(exonId)
    # 只要exonSet不为空集, 就随机提取exonSet中的一个exonId, 寻找其TES上下游各EXONRANGE范围内的exon
    while len(exonSet) != 0:
        exonId = exonSet.pop()
        exonObject = total.exonDict[exonId]
        chr = exonObject.chr
        TES = exonObject.TES
        # 声明list变量temp, [(<exonId>, <expression>), ...]
        temp = []  # [(<exonId>, <expression>), ...]
        # 寻找所有与该exon的TES相同的exon
        for exonId in indexTES[chr][TES].values():
            temp = temp + [(exonId, numpy.mean([total.exonDict[exonId].cellLineExpression.get(cellLine, 0) for cellLine in total.celllineInfo.keys()]))]
        # 递减寻找EXONRANGE范围内的exon
        temp = temp + findId(total=total, index=indexTES, chr=chr, location=TES, rangeMax=EXONRANGE, orientation='-', allowType="exon")
        # 递增寻找EXONRANGE范围内的exon
        temp = temp + findId(total=total, index=indexTES, chr=chr, location=TES, rangeMax=EXONRANGE, orientation='+', allowType="exon")
        # 去重
        temp = list(set(temp))
        # 判断是否寻找到了其他的exon
        if len(temp)==1:
            # 未寻找到其他的exon, 认为该exon的end无法修改, 跳过该exon
            exonSet.discard(temp[0][0])
            continue
        else:
            # 已寻找到其他的exon, 继续
            pass
        # 判断在temp中是否存在已校正端
        corSet = set()  # {(exonId, TES), ...}
        nonCorSet = set()  # {(exonId, TES), ...}
        # 分离其中的未校正端与已校正端
        for (exonId, expression) in temp:
            status = total.exonDict[exonId].status
            TES = total.exonDict[exonId].TES
            if status=="KNOWN" or "-TES" in status:
                # TES端可信
                corSet.add((exonId, TES))
            else:
                # TES端不可信
                nonCorSet.add((exonId, TES))
        if len(corSet) != 0:
            # 存在已校正端
            # 对每一个未校正端选择其距离最近的已校正端作为参考位点进行校正
            for (exonId, TES) in nonCorSet:
                # 寻找最近的已校正端
                diffTES = [(exonIdTemp, tempTES, abs(TES-tempTES)) for (exonIdTemp, tempTES) in corSet]
                diffTES = sorted(diffTES, key=lambda x: x[2])
                targetExonId = diffTES[0][0]
                targetTES = diffTES[0][1]
                # 修改TES, start/end位置
                total.exonDict[exonId].TES = targetTES
                if total.exonDict[exonId].strand == '+':
                    total.exonDict[exonId].end = targetTES
                else:
                    total.exonDict[exonId].start =targetTES
                # 改变status
                total.exonDict[exonId].status = total.exonDict[exonId].status + "-TES"
                # 删除exonSet中该exonId
                exonSet.discard(exonId)
                # 记录这一次校正
                exonCorrected["integrate TES"][exonId] = targetExonId
            # 结束此次循环
            continue
        else:
            # 不存在已校正端, 进行下一步
            pass
        # 以expression最大的exon为参照, 修改其他exon的TES
        exonIdTarget = None
        expressionMax = 0
        # 寻找expression最大的exon
        for (exonId, expression) in temp:
            if expression >= expressionMax:
                expressionMax = expression
                exonIdTarget = exonId
        # 修改其他的exon
        for (exonId, expression) in temp:
            # 修改其他exon的TES, start/end
            total.exonDict[exonId].TES = total.exonDict[exonIdTarget].TES
            if total.exonDict[exonId].strand == '+':
                total.exonDict[exonId].end = total.exonDict[exonIdTarget].end
            else:
                total.exonDict[exonId].start = total.exonDict[exonIdTarget].start
            # 修改status
            # 不应该修改status, 因为无已校正TSS作为参考, 所以TSS即使进行了合并也并不是可靠的
            #total.exonDict[exonId].status = total.exonDict[exonId].status + "-TES"
            # 删除exonSet中的这些exon
            exonSet.discard(exonId)
            # 记录这一次校正
            exonCorrected["integrate TES"][exonId] = exonIdTarget

# 统计TSS或TES已进行合并的exon的数量
infoDict["exon integrate TSS"] = len(exonCorrected["integrate TSS"])
infoDict["exon integrate TES"] = len(exonCorrected["integrate TSS"])

In [18]:
# 更新exon的映射并合并重复的gene, transcript, exon并重新计算表达量
total.refresh()
total.reIndex()

# 重新计算exon, transcript, gene的相对表达量
total.computeRelativeExpression()
total.computeCellLineExpression()

# 整理exon的status
for exonObject in total.exonDict.values():
    status = exonObject.status
    if status == "CORRECTED-TES-TSS":
        exonObject.status = "CORRECTED-TSS-TES"
    elif status == "NOVEL-TSS-TES":
        exonObject.status = "CORRECTED-TSS-TES"
    elif status == "NOVEL-TES-TSS":
        exonObject.status = "CORRECTED-TSS-TES"
    elif status == "NOVEL-TSS":
        exonObject.status = "CORRECTED-TSS"
    elif status == "NOVEL-TES":
        exonObject.status = "CORRECTED-TES"

# 统计exon合并后exon及transcript的数量
infoDict["total exon num after exon integrate"] = len(total.exonDict)
num=0
for exonId, exonObject in total.exonDict.items():
    if exonObject.status == "NOVEL":
        num+=1
infoDict["novel exon num after exon integrate"] = num
num = 0
for geneId, geneObject in total.geneDict.items():
    for transcriptId in geneObject.transcriptDict.keys():
        num += 1
infoDict["total transcript num after exon integrate"] = num
num = 0
for geneId, geneObject in total.geneDict.items():
    for transcriptObject in geneObject.transcriptDict.values():
        if transcriptObject.status == "NOVEL":
            num +=1
infoDict["novel transcript num after exon integrate"] = num

In [20]:
with open("10002_debug.pkl", "wb") as file:
    pickle.dump(total, file)

In [25]:
with open("10002_debug.pkl", "rb") as file:
    total = pickle.load(file)

In [22]:
# 打印exon, transcript数量变化信息
infoDict

{'total exon num before exon correct': 323363,
 'novel exon num before exon correct': 133311,
 'total transcript num before exon correct': 213375,
 'novel transcript num before exon correct': 185969,
 'exon mapped': 12094,
 'exon TSS-TES': 252,
 'exon TSS': 21082,
 'exon TES': 24244,
 'total exon num after exon correct': 317283,
 'novel exon num after exon correct': 66132,
 'total transcript num after exon correct': 211209,
 'novel transcript num after exon correct': 183803,
 'integrate TSS': 0,
 'integrate TES': 0,
 'exon integrate TSS': 23660,
 'exon integrate TES': 23660,
 'total exon num after exon integrate': 309587,
 'novel exon num after exon integrate': 60670,
 'total transcript num after exon integrate': 211121,
 'novel transcript num after exon integrate': 183715}

In [53]:
# 检查是否使用了错误的annotation文件
'''
检查是否存在部分transcript, 在total中该transcript属于geneTotal, 但在基因组注释中属于geneRef
可能的原因: 使用了不同版本的基因组注释信息

现在认为这种transcript不需要进行修改, 原修改过程:
需要将该transcript所属的gene由geneTotal映射到geneRef
    查询geneRef是否已记录
        若geneRefId已记录
            查询该transcript是否已存在于geneRef对象中
                若存在, 则合并这两个transcript
                若不存在
                    geneRef中新增该transcript对象
                    geneTotal中删除该transcript对象
        若geneRefId未记录但geneRef已记录
            获取其映射的geneRefMapped
            查询该transcript是否已存在于geneRefMapped对象中
                若存在, 则合并这两个transcript
                若不存在
                    在geneRef中新增该transcript对象
                    在geneTotal中删除该transcript对象
        若geneRef未记录
            记录geneRef对象
            在geneRef中新增该transcript对象
            删除geneTotal中的transcript对象
去除不包含transcript的gene
重建index
保存这种错误映射的数据到tsv中
'''
totalMap = {}
for geneId, geneObject in total.geneDict.items():
    for transcriptId, transcriptObject in geneObject.transcriptDict.items():
        totalMap[transcriptId] = geneId
annotationMap = {}
for transcriptId, transcriptDict in transcriptRef.items():
    geneId = transcriptDict["geneId"]
    annotationMap[transcriptId] = geneId
# 筛选出映射到错误gene的transcript
errorMap = {}  # {<transcriptId>: {"total": <geneId>, "ref": <geneId>}}
for transcriptId, geneId in totalMap.items():
    marker = annotationMap.get(transcriptId, None)
    if marker is None:
        continue
    if geneId != marker:
        errorMap[transcriptId] = {"total": geneId, "ref": marker}

if errorMap:
    print("[Warning]You might use the wrong annotation file, please check the version of annotation")

"""# 纠正错误映射
for transcriptId, d in errorMap.items():
    geneIdTotal = d["total"]
    geneIdRef = d["ref"]
    # 查询geneRef是否已记录
    (marker, mappedId) = total._geneCheck(geneId=geneIdRef, chr=geneRef[geneIdRef]["chr"], strand=geneRef[geneIdRef]["strand"], start=geneRef[geneIdRef]["start"], end=geneRef[geneIdRef]["end"])
    if marker == 0:
        # geneRefId已记录
        # 查询该transcript是否已存在于geneRef对象中
        if transcriptId in total.geneDict[geneIdRef].transcriptDict:
            # 存在, 合并这两个transcript
            total._transcriptMerge(transcriptId1=transcriptId, geneId1=geneIdRef,
                                   transcriptId2=transcriptId, geneId2=geneIdTotal)
        else:
            # 不存在
            # geneRef中新增该transcript对象
            
            # geneTotal中删除该transcript对象
    elif marker == 3:
        # geneRefId未记录但geneRef已记录
        # 查询该transcript是否已存在于geneRefMapped对象中
        if transcriptId in total.geneDict[mappedId].transcriptDict:
            # 该transcript对象已存在于geneIdRef中, 合并这两个transcript
            total._transcriptMerge(transcriptId1=transcriptId, geneId1=mappedId,
                                   transcriptId2=transcriptId, geneId2=geneIdTotal)
        else:
            # 该transcript对象未存在于geneIdRef中
            # geneRef中新增该transcript对象
            total.geneDict[geneIdRef].transcriptDict[transcriptId] = total.geneDict[geneIdTotal].transcriptDict[transcriptId]
            # geneTotal中删除该transcript对象
            total.geneDict[geneIdTotal].transcriptDict.pop(transcriptId)
    else:
        '''# geneRef未记录
        # 记录geneRef对象
        # 判断geneRef是否已记录
        total.geneAdd(status="KNOWN", chr=geneRef[geneIdRef]["chr"], strand=geneRef[geneIdRef]["strand"],
                      start=geneRef[geneIdRef]["start"], end=geneRef[geneIdRef]["end"], geneId=geneRef[geneIdRef]["geneId"],
                      geneVersion=geneRef[geneIdRef]["geneVersion"], geneName=geneRef[geneIdRef]["geneName"], geneBiotype=geneRef[geneIdRef]["geneBiotype"])
        # 在geneRef中新增该transcript对象
        total.geneDict[geneIdRef].transcriptDict[transcriptId] = total.geneDict[geneIdTotal].transcriptDict[transcriptId]
        # 删除geneTotal中的transcript对象
        total.geneDict[geneIdTotal].transcriptDict.pop(transcriptId)'''
        pass"""

'# 纠正错误映射\nfor transcriptId, d in errorMap.items():\n    geneIdTotal = d["total"]\n    geneIdRef = d["ref"]\n    # 查询geneRef是否已记录\n    (marker, mappedId) = total._geneCheck(geneId=geneIdRef, chr=geneRef[geneIdRef]["chr"], strand=geneRef[geneIdRef]["strand"], start=geneRef[geneIdRef]["start"], end=geneRef[geneIdRef]["end"])\n    if marker == 0:\n        # geneRefId已记录\n        # 查询该transcript是否已存在于geneRef对象中\n        if transcriptId in total.geneDict[geneIdRef].transcriptDict:\n            # 存在, 合并这两个transcript\n            total._transcriptMerge(transcriptId1=transcriptId, geneId1=geneIdRef,\n                                   transcriptId2=transcriptId, geneId2=geneIdTotal)\n        else:\n            # 不存在\n            # geneRef中新增该transcript对象\n            \n            # geneTotal中删除该transcript对象\n    elif marker == 3:\n        # geneRefId未记录但geneRef已记录\n        # 查询该transcript是否已存在于geneRefMapped对象中\n        if transcriptId in total.geneDict[mappedId].transcriptDict:\n            #

In [19]:
# 声明变量
transcriptSet = set()  # {(<geneId>, <transcriptId>), ...}
transcriptCorrected = {"mapped": {}, "TSS-TES": {}, "TSS": {}, "TES": {}, "integrate TSS": {}, "integrate TES": {}}
# 准备transcriptId与geneId的映射关系
transcriptGeneMap = {}
for geneId, geneObject in total.geneDict.items():
    for transcriptId, transcriptObject in geneObject.transcriptDict.items():
        transcriptGeneMap[transcriptId] = geneId
transcriptGeneMapKeys = set(transcriptGeneMap.keys())
# 准备total中transcript的index
totalTranscriptIndex = total.transcriptIndexBuild(siteType="TSS")

In [20]:
# 统计校正前transcript的数量
num = 0
for geneObject in total.geneDict.values():
    for transcriptObject in geneObject.transcriptDict.values():
        num += 1
infoDict["total transcript num before transcript correct"] = num
num = 0
for geneObject in total.geneDict.values():
    for transcriptObject in geneObject.transcriptDict.values():
        if transcriptObject.status == "NOVEL":
            num += 1
infoDict["novel transcript num before transcript correct"] = num

In [21]:
# 遍历所有status为KNOWN的transcript, 校正其TSS及TES端
for geneId, geneObject in total.geneDict.items():
    for transcriptId, transcriptObject in geneObject.transcriptDict.items():
        status = transcriptObject.status
        if status == "KNOWN":
            # 根据transcriptId, 在transcriptRef中检索
            # 修改其TSS/TES, start/end
            transcriptObject.start = transcriptRef[transcriptId]["start"]
            transcriptObject.end = transcriptRef[transcriptId]["end"]
            transcriptObject.TSS = transcriptRef[transcriptId]["TSS"]
            transcriptObject.TES = transcriptRef[transcriptId]["TES"]

In [22]:
# 遍历所有status为NOVEL的transcript, 将所有transcriptId存储至变量transcriptSet中
transcriptSet = set()
for geneId, geneObject in total.geneDict.items():
    for transcriptId, transcriptObject in geneObject.transcriptDict.items():
        status = transcriptObject.status
        if status == "NOVEL":
            transcriptSet.add((geneId, transcriptId))

In [23]:
# 根据基因组注释校正transcript
for (geneId, transcriptId) in tqdm.tqdm(transcriptSet, leave=True, desc="transcript"):
    # 寻找transcriptSet中可映射的transcript
    chr = total.geneDict[geneId].chr
    transcriptObject = total.geneDict[geneId].transcriptDict[transcriptId]
    TSS = transcriptObject.TSS
    TES = transcriptObject.TES
    exonSet = set(transcriptObject.exonList)
    # 存储该transcript所能映射的所有参考transcript
    transcriptList = []  # [[<transcriptRefId>, <num>, <bool>, <bool>], ...]
    # 根据TSS获取其上下游rangeTSS范围内的transcript
    temp = [transcriptIndex["TSS"][chr].get(i, None) for i in range(TSS-RANGETSS, TSS+RANGETSS+1)]
    # 去除空值
    while None in temp:
        temp.remove(None)
    while {} in temp:
        temp.remove({})
        pass
    # 拆解list
    if temp:
        temp = [{tempTES: tempId} for d in temp for tempTES, tempIdList in d.items() for tempId in tempIdList]
    for d in temp:
        for tempTES, tempTranscriptId in d.items():
            if abs(tempTES-TES) <= RANGETES:
                # 从中筛选出[TES-RANGETES, TES+RANGETES]范围的所有transcript, 认为这些transcript为可双端映射的transcript
                num = abs(TSS-transcriptRef[tempTranscriptId]["TSS"]) + abs(TES-tempTES)
                transcriptList.append((tempTranscriptId, num, True, True))
            else:
                # 仅TSS可映射到tempTranscript
                num = abs(TSS-transcriptRef[tempTranscriptId]["TSS"])
                transcriptList.append((tempTranscriptId, num, True, False))
    # 根据TES获取其上下游rangeTES范围内的transcript
    temp = [transcriptIndex["TES"][chr].get(i, None) for i in range(TES-RANGETES, TES+RANGETES+1)]
    # 去除空值
    while None in temp:
        temp.remove(None)
    while {} in temp:
        temp.remove({})
        pass
    # 拆解list
    if temp:
        temp = [{tempTES: tempId} for d in temp  for tempTES, tempIdList in d.items() for tempId in tempIdList]
    for d in temp:
        for tempTSS, tempTranscriptId in d.items():
            # 从中筛选出(-oo, TSS-RANGETSS)U(TSS+RANGETSS, +oo)的所有transcript, 认为这些transcript为仅可映射到TES的transcript
            if abs(tempTSS-TSS) > RANGETSS:
                # 仅TES可映射到tempTranscript
                num = abs(TES-transcriptRef[tempTranscriptId]["TES"])
                transcriptList.append((tempTranscriptId, num, False, True))

    # 准备映射
    # 判断是否存在映射
    if not transcriptList:
        # 不存在映射, 跳过该transcript
        continue
    # 判断映射的类型
    # 判断是否仅一个映射
    if len(transcriptList) == 1:
        # 仅单个映射仅单个映射
        (transcriptRefId, num, markerTSS, markerTES) = transcriptList[0]
        transcriptRefSet = {(transcriptRefId, num, markerTSS, markerTES, None, None)}
    else:
        # 优先筛选出双端映射的transcript
        doubleMapped = []
        for (transcriptRefId, num, markerTSS, markerTES) in transcriptList:
            if markerTSS == markerTES == True:
                doubleMapped.append([transcriptRefId, num, markerTSS, markerTES])
        # 判断是否存在双端映射的transcript
        if doubleMapped:
            # 存在双端映射的transcript
            doubleMapped = sorted(doubleMapped, key=lambda x: x[1])
            # 不存在transcriptId相同的项
            # 筛选出exon组成最相似的transcript
            # exon组成相似程度 = (transcriptRef的exon数量 与 transcript的exon数量 中最小的数/最大的数) * transcript与transcriptRef重叠的exon数量
            exonCombinationValueList = []
            for [transcriptRefId, num, markerTSS, markerTES] in doubleMapped:
                exonTranscript = set(total.geneDict[geneId].transcriptDict[transcriptId].exonList)
                exonTranscriptRef = transcriptRef[transcriptRefId]["exonSet"]
                (numMin, numMax) = sorted([len(exonTranscript), len(exonTranscriptRef)])
                exonValue = (numMin / numMax) * len(exonTranscript.intersection(exonTranscriptRef))
                exonCombinationValueList.append([transcriptRefId, num, exonValue])
            # 若exon组成相似程度相同, 则筛选出TSS与TES相差碱基数最少的transcript
            exonCombinationValueList = sorted(exonCombinationValueList, key=lambda x: (-x[2], x[1]))
            transcriptRefSet = {(transcriptRefId, num, markerTSS, markerTES, None, None) for [transcriptRefId, num, markerTSS, markerTES] in doubleMapped if transcriptRefId==exonCombinationValueList[0][0]}
        else:
            # 不存在双端映射的transcript
            transcriptRefSet=set()  # 可能有一个元素, 也可能有两个元素
            # TSS端
            transcriptTSSList = [(transcriptRefId, num, markerTSS, markerTES) for (transcriptRefId, num, markerTSS, markerTES) in transcriptList if markerTSS is True]
            # 不存在transcriptId相同的项
            # 筛选出exon组成最相似的transcript
            # exon组成相似程度 = (transcriptRef的exon数量 与 transcript的exon数量 中最小的数/最大的数) * transcript与transcriptRef重叠的exon数量
            exonCombinationValueList = []
            for [transcriptRefId, num, markerTSS, markerTES] in transcriptTSSList:
                exonTranscript = set(total.geneDict[geneId].transcriptDict[transcriptId].exonList)
                exonTranscriptRef = transcriptRef[transcriptRefId]["exonSet"]
                (numMin, numMax) = sorted([len(exonTranscript), len(exonTranscriptRef)])
                exonValue = (numMin / numMax) * len(exonTranscript.intersection(exonTranscriptRef))
                exonCombinationValueList.append([transcriptRefId, num, exonValue])
            # 若exon组成相似程度相同, 则筛选出TSS与TES相差碱基数最少的transcript
            exonCombinationValueList = sorted(exonCombinationValueList, key=lambda x: (-x[2], x[1]))
            transcriptRefSet = transcriptRefSet.union({(transcriptRefId, num, markerTSS, markerTES, None, None) for [transcriptRefId, num, markerTSS, markerTES] in transcriptTSSList if transcriptRefId==exonCombinationValueList[0][0]})
            # TES端
            transcriptTESList = [(transcriptRefId, num, markerTSS, markerTES) for (transcriptRefId, num, markerTSS, markerTES) in transcriptList if markerTES is True]
            # 不存在transcriptId相同的项
            # 其次筛选出exon组成最相似的transcript
            # exon组成相似程度=参考transcript与该transcript的exon的重叠数量
            exonCombinationValueList = []
            for [transcriptRefId, num, markerTSS, markerTES] in transcriptTESList:
                exonTranscript = set(total.geneDict[geneId].transcriptDict[transcriptId].exonList)
                exonTranscriptRef = transcriptRef[transcriptRefId]["exonSet"]
                (numMin, numMax) = sorted([len(exonTranscript), len(exonTranscriptRef)])
                exonValue = (numMin / numMax) * len(exonTranscript.intersection(exonTranscriptRef))
                exonCombinationValueList.append([transcriptRefId, num, exonValue])
            # 若exon组成相似程度相同, 则筛选出TSS与TES相差碱基数最少的transcript
            exonCombinationValueList = sorted(exonCombinationValueList, key=lambda x: (-x[2], x[1]))
            transcriptRefSet = transcriptRefSet.union({(transcriptRefId, num, markerTSS, markerTES, None, None) for [transcriptRefId, num, markerTSS, markerTES] in transcriptTESList if transcriptRefId==exonCombinationValueList[0][0]})
    # 检查transcriptRef是否已记录
    temp = [[transcriptRefId, num, markerTSS, markerTES, marker, mapped] for (transcriptRefId, num, markerTSS, markerTES, marker, mapped) in transcriptRefSet]
    for index, value in enumerate(temp):
        transcriptRefId = value[0]
        if transcriptRefId in transcriptGeneMapKeys:
            # transcriptRefId已记录, 认为该transcript已记录
            temp[index][4] = 1
            temp[index][5] = None
        else:
            # 检查totalTranscriptIndex中是否已记录该transcript
            transcriptRefChr = transcriptRef[transcriptRefId]["chr"]
            transcriptRefTSS = transcriptRef[transcriptRefId]["TSS"]
            transcriptRefTES = transcriptRef[transcriptRefId]["TES"]
            transcriptRefExonSet = transcriptRef[transcriptRefId]["exonSet"]
            if transcriptRefTSS not in totalTranscriptIndex[transcriptRefChr].keys():
                # 未记录该transcript
                temp[index][4] = 3
                temp[index][5] = None
            else:
                if transcriptRefTES not in totalTranscriptIndex[transcriptRefChr][transcriptRefTSS].keys():
                    # 未记录该transcript
                    temp[index][4] = 3
                    temp[index][5] = None
                else:
                    markerBreak = False
                    transcriptIdListTemp = totalTranscriptIndex[transcriptRefChr][transcriptRefTSS][transcriptRefTES]
                    for transcriptIdTemp in transcriptIdListTemp:
                        transcriptTempExon = set(total.geneDict[transcriptGeneMap[transcriptIdTemp]].transcriptDict[transcriptIdTemp].exonList)
                        if transcriptTempExon == transcriptRefExonSet:
                            # 已记录该transcript
                            markerBreak = True
                            # 判断已记录的那个transcript是否为NOVEL类型的transcript
                            if total.geneDict[transcriptGeneMap[transcriptIdTemp]].transcriptDict[transcriptIdTemp].status == "NOVEL":
                                temp[index][4] = 3
                                temp[index][5] = None
                            else:
                                temp[index][4] = 2
                                temp[index][5] = transcriptIdTemp
                            break
                    if markerBreak is True:
                        pass
                    else:
                        # 未记录该transcript
                        temp[index][4] = 3
                        temp[index][5] = None
    transcriptRefSet = {(transcriptRefId, num, markerTSS, markerTES, marker, mapped) for [transcriptRefId, num, markerTSS, markerTES, marker, mapped] in temp}
    # 开始映射
    if len(transcriptRefSet) == 2:
        # 分别映射到两transcript
        transcriptCorrected["TSS-TES"][transcriptId] = {"TSS": None, "TES": None}
        oldTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
        oldTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
        # 对transcript进行校正
        total.geneDict[geneId].transcriptDict[transcriptId].status = "CORRECTED-TSS-TES"
        for (transcriptRefId, num, markerTSS, markerTES, marker, mapped) in transcriptRefSet:
            if markerTSS is True:
                # 修改TSS, start/end
                total.geneDict[geneId].transcriptDict[transcriptId].TSS = transcriptRef[transcriptRefId]["TSS"]
                if total.geneDict[geneId].strand == '+':
                    total.geneDict[geneId].transcriptDict[transcriptId].start = transcriptRef[transcriptRefId]["start"]
                else:
                    total.geneDict[geneId].transcriptDict[transcriptId].end = transcriptRef[transcriptRefId]["end"]
                transcriptCorrected["TSS-TES"][transcriptId]["TSS"] = transcriptRefId
            elif markerTES is True:
                # 修改TES, start/end
                total.geneDict[geneId].transcriptDict[transcriptId].TES = transcriptRef[transcriptRefId]["TES"]
                if total.geneDict[geneId].strand == '+':
                    total.geneDict[geneId].transcriptDict[transcriptId].end = transcriptRef[transcriptRefId]["end"]
                else:
                    total.geneDict[geneId].transcriptDict[transcriptId].start = transcriptRef[transcriptRefId]["start"]
                transcriptCorrected["TSS-TES"][transcriptId]["TES"] = transcriptRefId
        # 改变totalTranscriptIndex中该transcript的位置信息
        newTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
        newTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
        chr = total.geneDict[geneId].chr
        totalTranscriptIndex[chr][oldTSS][oldTES] = [i for i in totalTranscriptIndex[chr][oldTSS][oldTES] if i!=transcriptId]
        if not totalTranscriptIndex[chr][oldTSS][oldTES]:
            totalTranscriptIndex[chr][oldTSS].pop(oldTES)
        if not totalTranscriptIndex[chr][oldTSS]:
            totalTranscriptIndex[chr].pop(oldTSS)
        if newTSS not in totalTranscriptIndex[chr]:
            totalTranscriptIndex[chr][newTSS] = {newTES: [transcriptId]}
        else:
            if newTES not in totalTranscriptIndex[chr][newTSS]:
                totalTranscriptIndex[chr][newTSS][newTES] = [transcriptId]
            else:
                totalTranscriptIndex[chr][newTSS][newTES].append(transcriptId)
    else:
        # 映射到单个transcript
        (transcriptRefId, num, markerTSS, markerTES, marker, mapped) = transcriptRefSet.pop()
        if markerTSS == markerTES == True:
            # 双端映射
            # 判断transcript与transcriptRef的exon组成是否一致
            transcriptExonSet = set(total.geneDict[geneId].transcriptDict[transcriptId].exonList)
            transcriptRefExonSet = transcriptRef[transcriptRefId]["exonSet"]
            if transcriptExonSet == transcriptRefExonSet:
                # exon组成一致, 可以将transcript合并至transcriptRef
                integrateMarker = True
            else:
                # exon组成不一致, 不可将transcript合并至transcriptRef
                integrateMarker = False
            if integrateMarker is True:
                # 可合并
                if marker == 1:
                    # transcriptRefId已记录
                    geneRefId = transcriptRef[transcriptRefId]["geneId"]
                    refChr = total.geneDict[geneRefId].chr
                    refTSS = total.geneDict[geneRefId].transcriptDict[transcriptRefId].TSS
                    refTES = total.geneDict[geneRefId].transcriptDict[transcriptRefId].TES
                    idChr = total.geneDict[geneId].chr
                    idTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
                    idTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
                    # 将transcript合并至transcriptRef
                    (deletedId, remainedId) = total._transcriptMerge(transcriptId1=transcriptRefId,
                                                                     transcriptId2=transcriptId,
                                                                     geneId1=geneRefId,
                                                                     geneId2=geneId)
                    (tempChr, tempTSS, tempTES) = ((refChr, refTSS, refTES), (idChr, idTSS, idTES))[deletedId == transcriptId]
                    # 删除totalTranscriptIndex中的transcript的相关信息
                    totalTranscriptIndex[tempChr][tempTSS][tempTES] = [i for i in totalTranscriptIndex[tempChr][tempTSS][tempTES] if i!=deletedId]
                    if not totalTranscriptIndex[tempChr][tempTSS][tempTES]:
                        totalTranscriptIndex[tempChr][tempTSS].pop(tempTES)
                    if not totalTranscriptIndex[tempChr][tempTSS]:
                        totalTranscriptIndex[tempChr].pop(tempTSS)
                elif marker == 2:
                    # transcriptRef已记录, id为mapped
                    geneRefId = transcriptRef[mapped]["geneId"]
                    refChr = total.geneDict[geneRefId].chr
                    refTSS = total.geneDict[geneRefId].transcriptDict[mapped].TSS
                    refTES = total.geneDict[geneRefId].transcriptDict[mapped].TES
                    idChr = total.geneDict[geneId].chr
                    idTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
                    idTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
                    # 将transcript合并至transcriptRef
                    (deletedId, remainedId) = total._transcriptMerge(transcriptId1=mapped,
                                                                     transcriptId2=transcriptId,
                                                                     geneId1=geneRefId,
                                                                     geneId2=geneId)
                    (tempChr, tempTSS, tempTES) = ((refChr, refTSS, refTES), (idChr, idTSS, idTES))[deletedId == transcriptId]
                    # 删除totalTranscriptIndex中的transcript的相关信息
                    totalTranscriptIndex[tempChr][tempTSS][tempTES] = [i for i in totalTranscriptIndex[tempChr][tempTSS][tempTES] if i!=deletedId]
                    if not totalTranscriptIndex[tempChr][tempTSS][tempTES]:
                        totalTranscriptIndex[tempChr][tempTSS].pop(tempTES)
                    if not totalTranscriptIndex[tempChr][tempTSS]:
                        totalTranscriptIndex[tempChr].pop(tempTSS)
                elif marker == 3:
                    # transcriptRef未记录
                    geneRefId = transcriptRef[transcriptRefId]["geneId"]
                    # 检查transcriptRef所属的geneRef是否已记录
                    (tempMarker, tempMapped) = total._geneCheck(geneId=geneRefId, chr=geneRef[geneRefId]["chr"], strand=geneRef[geneRefId]["strand"], start=geneRef[geneRefId]["start"], end=geneRef[geneRefId]["end"])
                    if geneRefId in total.geneExisted:
                        # 已映射, 认为geneRef已记录
                        geneRefId = total.geneExisted[geneRefId]
                    elif tempMarker == 0:
                        # geneRefId已记录
                        pass
                    elif tempMarker == 3:
                        # geneRef可映射到tempMapped
                        geneRefId = tempMapped
                    else:
                        # 未映射, 添加该gene对象
                        total.geneAdd(status="KNOWN",
                                      chr=geneRef[geneRefId]["chr"],
                                      strand=geneRef[geneRefId]["strand"],
                                      start=geneRef[geneRefId]["start"],
                                      end=geneRef[geneRefId]["end"],
                                      geneId=geneRef[geneRefId]["geneId"],
                                      geneVersion=geneRef[geneRefId]["geneVersion"],
                                      geneName=geneRef[geneRefId]["geneName"],
                                      geneBiotype=geneRef[geneRefId]["geneBiotype"])
                    # 在对应的geneRefObject中添加transcriptRef对象
                    total.geneDict[geneRefId].transcriptDict[transcriptRefId] = Transcript(status="KNOWN",
                                                                                           transcriptId=transcriptRefId,
                                                                                           transcriptVersion=transcriptRef[transcriptRefId]["transcriptVersion"],
                                                                                           transcriptName=transcriptRef[transcriptRefId]["transcriptName"],
                                                                                           transcriptBiotype=transcriptRef[transcriptRefId]["transcriptBiotype"],
                                                                                           start=transcriptRef[transcriptRefId]["start"],
                                                                                           end=transcriptRef[transcriptRefId]["end"],
                                                                                           exonList=list(transcriptRef[transcriptRefId]["exonSet"]))
                    # 在transcriptGeneMap中添加transcriptRefId: geneRefId的映射
                    transcriptGeneMap[transcriptRefId] = geneRefId
                    transcriptGeneMapKeys.add(transcriptRefId)
                    # 合并transcript至transcriptRef
                    refChr = total.geneDict[geneRefId].chr
                    refTSS = total.geneDict[geneRefId].transcriptDict[transcriptRefId].TSS
                    refTES = total.geneDict[geneRefId].transcriptDict[transcriptRefId].TES
                    idChr = total.geneDict[geneId].chr
                    idTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
                    idTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
                    (deletedId, remainedId) = total._transcriptMerge(transcriptId1=transcriptRefId,
                                                                     transcriptId2=transcriptId,
                                                                     geneId1=geneRefId,
                                                                     geneId2=geneId)
                    if deletedId == transcriptId:
                        (tempChr, tempTSS, tempTES) = (idChr, idTSS, idTES)
                        (remainedChr, remainedTSS, remainedTES) = (refChr, refTSS, refTES)
                    else:
                        (tempChr, tempTSS, tempTES) = (refChr, refTSS, refTES)
                        (remainedChr, remainedTSS, remainedTES) = (idChr, idTSS, idTES)
                    # 删除totalTranscriptIndex中的transcript的相关信息
                    totalTranscriptIndex[tempChr][tempTSS][tempTES] = [i for i in totalTranscriptIndex[tempChr][tempTSS][tempTES] if i!=deletedId]
                    if not totalTranscriptIndex[tempChr][tempTSS][tempTES]:
                        totalTranscriptIndex[tempChr][tempTSS].pop(tempTES)
                    if not totalTranscriptIndex[tempChr][tempTSS]:
                        totalTranscriptIndex[tempChr].pop(tempTSS)
                    # 添加totalTranscriptIndex中的transcriptRef的相关信息
                    if remainedTSS not in totalTranscriptIndex[remainedChr]:
                        totalTranscriptIndex[remainedChr][remainedTSS] = {remainedTES: [remainedId]}
                    else:
                        if remainedTES not in totalTranscriptIndex[remainedChr][remainedTSS]:
                            totalTranscriptIndex[remainedChr][remainedTSS][remainedTES] = [remainedId]
                        else:
                            totalTranscriptIndex[remainedChr][remainedTSS][remainedTES].append(remainedId)
                else:
                    raise ValueError("marker should be 1 or 2 or 3, but it is {}".format(marker))
                # 记录这一次合并到transcriptCorrected["mapped"]
                transcriptCorrected["mapped"][transcriptId] = transcriptRefId
            else:
                # 不可合并
                oldTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
                oldTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
                # 根据transcriptRef修改transcript的start, end, TSS, TES
                total.geneDict[geneId].transcriptDict[transcriptId].start = transcriptRef[transcriptRefId]["start"]
                total.geneDict[geneId].transcriptDict[transcriptId].end = transcriptRef[transcriptRefId]["end"]
                total.geneDict[geneId].transcriptDict[transcriptId].TSS = transcriptRef[transcriptRefId]["TSS"]
                total.geneDict[geneId].transcriptDict[transcriptId].TES = transcriptRef[transcriptRefId]["TES"]
                # 修改transcript的status为"CORRECTED-TSS-TES"
                total.geneDict[geneId].transcriptDict[transcriptId].status = "CORRECTED-TSS-TES"
                # 记录该映射到transcriptCorrected["TSS-TES"]
                transcriptCorrected["TSS-TES"][transcriptId] = {"TSS": transcriptRef[transcriptRefId]["TSS"], "TES": transcriptRef[transcriptRefId]["TES"]}
                # 改变totalTranscriptIndex中该transcript的位置信息
                newTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
                newTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
                chr = total.geneDict[geneId].chr
                totalTranscriptIndex[chr][oldTSS][oldTES] = [i for i in totalTranscriptIndex[chr][oldTSS][oldTES] if i!=transcriptId]
                if not totalTranscriptIndex[chr][oldTSS][oldTES]:
                    totalTranscriptIndex[chr][oldTSS].pop(oldTES)
                if not totalTranscriptIndex[chr][oldTSS]:
                    totalTranscriptIndex[chr].pop(oldTSS)
                if newTSS not in totalTranscriptIndex[chr]:
                    totalTranscriptIndex[chr][newTSS] = {newTES: [transcriptId]}
                else:
                    if newTES not in totalTranscriptIndex[chr][newTSS]:
                        totalTranscriptIndex[chr][newTSS][newTES] = [transcriptId]
                    else:
                        totalTranscriptIndex[chr][newTSS][newTES].append(transcriptId)
        else:
            # 单端映射
            oldTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
            oldTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
            chr = total.geneDict[geneId].chr
            if markerTSS is True:
                # 仅TSS端可映射
                total.geneDict[geneId].transcriptDict[transcriptId].status = "CORRECTED-TSS"
                total.geneDict[geneId].transcriptDict[transcriptId].TSS = transcriptRef[transcriptRefId]["TSS"]
                transcriptCorrected["TSS"][transcriptId] = transcriptRefId
                if total.geneDict[geneId].strand == '+':
                    total.geneDict[geneId].transcriptDict[transcriptId].start = transcriptRef[transcriptRefId]["start"]
                else:
                    total.geneDict[geneId].transcriptDict[transcriptId].end = transcriptRef[transcriptRefId]["end"]
                # 修改transcript在totalTranscriptIndex中的位置信息
                newTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
                newTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
                totalTranscriptIndex[chr][oldTSS][oldTES] = [i for i in totalTranscriptIndex[chr][oldTSS][oldTES] if i!=transcriptId]
                if not totalTranscriptIndex[chr][oldTSS][oldTES]:
                    totalTranscriptIndex[chr][oldTSS].pop(oldTES)
                if not totalTranscriptIndex[chr][oldTSS]:
                    totalTranscriptIndex[chr].pop(oldTSS)
                if newTSS not in totalTranscriptIndex[chr]:
                    totalTranscriptIndex[chr][newTSS] = {newTES: [transcriptId]}
                else:
                    if newTES not in totalTranscriptIndex[chr][newTSS]:
                        totalTranscriptIndex[chr][newTSS][newTES] = [transcriptId]
                    else:
                        totalTranscriptIndex[chr][newTSS][newTES].append(transcriptId)
            else:
                # 仅TES端可映射是
                total.geneDict[geneId].transcriptDict[transcriptId].status = "CORRECTED-TES"
                total.geneDict[geneId].transcriptDict[transcriptId].TES = transcriptRef[transcriptRefId]["TES"]
                transcriptCorrected["TES"][transcriptId] = transcriptRefId
                if total.geneDict[geneId].strand == '+':
                    total.geneDict[geneId].transcriptDict[transcriptId].end = transcriptRef[transcriptRefId]["end"]
                else:
                    total.geneDict[geneId].transcriptDict[transcriptId].start = transcriptRef[transcriptRefId]["start"]
                # 修改transcript在totalTranscriptIndex中的位置信息
                newTSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
                newTES = total.geneDict[geneId].transcriptDict[transcriptId].TES
                totalTranscriptIndex[chr][oldTSS][oldTES] = [i for i in totalTranscriptIndex[chr][oldTSS][oldTES] if i!=transcriptId]
                if not totalTranscriptIndex[chr][oldTSS][oldTES]:
                    totalTranscriptIndex[chr][oldTSS].pop(oldTES)
                if not totalTranscriptIndex[chr][oldTSS]:
                    totalTranscriptIndex[chr].pop(oldTSS)
                if newTSS not in totalTranscriptIndex[chr]:
                    totalTranscriptIndex[chr][newTSS] = {newTES: [transcriptId]}
                else:
                    if newTES not in totalTranscriptIndex[chr][newTSS]:
                        totalTranscriptIndex[chr][newTSS][newTES] = [transcriptId]
                    else:
                        totalTranscriptIndex[chr][newTSS][newTES].append(transcriptId)

transcript: 100%|██████████| 183715/183715 [00:19<00:00, 9569.73it/s] 


In [24]:
total.refresh()
total.reIndex()
total.computeRelativeExpression()
total.computeCellLineExpression()

In [25]:
# # 统计校正后transcript的数量
num = 0
for geneObject in total.geneDict.values():
    for transcriptObject in geneObject.transcriptDict.values():
        num += 1
infoDict["total transcript num after transcript correct"] = num
num = 0
for geneObject in total.geneDict.values():
    for transcriptObject in geneObject.transcriptDict.values():
        if transcriptObject.status == "NOVEL":
            num += 1
infoDict["novel transcript num after transcript correct"] = num

In [26]:
# 统计transcriptCorrect保存的校正信息
num = {"KNOWN":0, "NOVEL":0, "CORRECTED-TSS":0, "CORRECTED-TES":0, "CORRECTED-TSS-TES":0}
for geneObject in total.geneDict.values():
    for transcriptObject in geneObject.transcriptDict.values():
        status = transcriptObject.status
        num[status] += 1
print(num)

{'KNOWN': 27433, 'NOVEL': 70887, 'CORRECTED-TSS': 16470, 'CORRECTED-TES': 38526, 'CORRECTED-TSS-TES': 57327}


In [27]:
# 声明变量
transcriptSet = set()  # {(geneId, transcriptId), ...}
transcriptIntegrate = {"TSS": {}, "TES": {}}  # {"TSS": {<transcriptId>: <TSSRef>, ...}, "TES": {<transcriptId>: <TESRef>, ...}}
transcriptGeneMap = {}  # {<transcriptId>: <geneId>, ...}
for geneId, geneObject in total.geneDict.items():
    for transcriptId, transcriptObject in geneObject.transcriptDict.items():
        transcriptGeneMap[transcriptId] = geneId

In [28]:
# 合并TSS
# 根据TSS建立index
indexTSS = total.transcriptIndexBuild(siteType="TSS")
# 筛选出TSS端不可信的transcript
transcriptSet = set()
for geneId, geneObject in total.geneDict.items():
    for transcriptId, transcriptObject in geneObject.transcriptDict.items():
        status = transcriptObject.status
        if status == "KNOWN" or "-TSS" in status:
            # 过滤掉TSS端可信的transcript
            continue
        else:
            transcriptSet.add((geneId, transcriptId))
# 只要transcriptSet不为空集, 就提取transcriptSet中的一个transcriptId
while transcriptSet:
    temp = []  # 存储该transcript与周围的transcript  [(<geneId>, <transcriptId>, <expression>), ...]
    (geneId, transcriptId) = transcriptSet.pop()
    chr = total.geneDict[geneId].chr
    transcriptObject = total.geneDict[geneId].transcriptDict[transcriptId]
    TSS = transcriptObject.TSS
    # 寻找其TSS上下游各RANGETSS范围内的transcript(不包括TSS)
    # 递减寻找RANGETSS范围内的transcript
    temp = temp + findId(total=total, index=indexTSS, chr=chr, location=TSS, rangeMax=RANGETSS, orientation='-', allowType="transcript", transcriptGeneMap=transcriptGeneMap)
    # 递增寻找RANGETSS范围内的transcript
    temp = temp + findId(total=total, index=indexTSS, chr=chr, location=TSS, rangeMax=RANGETSS, orientation='+', allowType="transcript", transcriptGeneMap=transcriptGeneMap)
    # 判断transcript附近是否存在transcriptAdjacent
    if temp:
        existedMarker = True
    else:
        existedMarker = False
        # transcript附近不存在transcriptAdjacent, 该transcript的TSS无法修改, 跳过该transcript
        continue
    # temp中添加所有与该transcript的TSS相同的transcriptAdjacent
    for transcriptIdList in indexTSS[chr][TSS].values():
        for transcriptId in transcriptIdList:
            geneId = transcriptGeneMap[transcriptId]
            temp = temp + [(geneId, transcriptId, numpy.mean([total.geneDict[geneId].transcriptDict[transcriptId].cellLineExpression.get(cellLine, 0) for cellLine in total.celllineInfo.keys()]))]
    # 去重, 虽然不应该存在重复
    temp = list(set(temp))
    # 判断在temp中是否存在已校正端
    # 分离其中的未校正端与已校正端
    corSet = set()  # {(geneId, transcriptId, TSS), ...}
    nonCorSet = set()  # {(geneId, transcriptId, TSS), ...}
    for (geneId, transcriptId, expression) in temp:
        status = total.geneDict[geneId].transcriptDict[transcriptId].status
        TSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
        if status=="KNOWN" or "-TSS" in status:
            # TSS端可信
            corSet.add((geneId, transcriptId, TSS))
        else:
            # TSS端不可信
            nonCorSet.add((geneId, transcriptId, TSS))
    if corSet:
        # 存在已校正端
        # 对每一个未校正端选择其距离最近的已校正端作为参考位点进行校正
        for (geneId, transcriptId, TSS) in nonCorSet:
            chr = total.geneDict[geneId].chr
            TES = total.geneDict[geneId].transcriptDict[transcriptId].TES
            # 寻找最近的已校正端
            diffTSS = [(geneIdTemp, transcriptIdTemp, abs(TSS-TSSTemp)) for (geneIdTemp, transcriptIdTemp, TSSTemp) in corSet]
            diffTSS = sorted(diffTSS, key=lambda x: x[2])
            (geneRefId, transcriptRefId, diffNum) = diffTSS[0]
            TSSRef = total.geneDict[geneRefId].transcriptDict[transcriptRefId].TSS
            # 修改TSS, start/end位置
            total.geneDict[geneId].transcriptDict[transcriptId].TSS = TSSRef
            if total.geneDict[geneId].strand == '+':
                total.geneDict[geneId].transcriptDict[transcriptId].start = TSSRef
            else:
                total.geneDict[geneId].transcriptDict[transcriptId].end = TSSRef
            # 修改indexTSS中相应transcript的位置
            # 在indexTSS中删除transcript的原位置
            indexTSS[chr][TSS][TES] = [i for i in indexTSS[chr][TSS][TES] if i!=transcriptId]
            if not indexTSS[chr][TSS][TES]:
                indexTSS[chr][TSS].pop(TES)
            if not indexTSS[chr][TSS]:
                indexTSS[chr].pop(TSS)
            # 在indexTSS中新增transcript的新位置
            if TSSRef not in indexTSS[chr]:
                indexTSS[chr][TSSRef] = {TES: [transcriptId]}
            else:
                if TES not in indexTSS[chr][TSSRef]:
                    indexTSS[chr][TSSRef][TES] = [transcriptId]
                else:
                    indexTSS[chr][TSSRef][TES].append(transcriptId)
            # transcriptIntegrate中添加该映射
            transcriptIntegrate["TSS"][transcriptId] = TSSRef
            # 改变status
            total.geneDict[geneId].transcriptDict[transcriptId].status = total.geneDict[geneId].transcriptDict[transcriptId].status + "-TSS"
            # 删除transcriptSet中该transcriptId
            transcriptSet.discard(transcriptId)
    else:
        # 不存在已校正端
        # 确定TSSRef
        # 建立TSS位点与transcript表达的关系, 选择表达量最高的TSS作为TSSRef
        TSSRef = {}
        for (geneId, transcriptId, expression) in temp:
            TSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
            TSSExpression = TSSRef.get(TSS, 0)
            TSSExpression = TSSExpression + expression
            TSSRef[TSS] = TSSExpression
        TSSRef = [(TSS, expression) for TSS, expression in TSSRef.items()]
        TSSRef = sorted(TSSRef, key=lambda x: x[1], reverse=True)
        TSSRef = TSSRef[0][0]
        # 修改temp中所有transcript的TSS, start/end
        for (geneId, transcriptId, expression) in temp:
            TSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
            TES = total.geneDict[geneId].transcriptDict[transcriptId].TES
            total.geneDict[geneId].transcriptDict[transcriptId].TSS = TSSRef
            if total.geneDict[geneId].strand == '+':
                total.geneDict[geneId].transcriptDict[transcriptId].start = TSSRef
            else:
                total.geneDict[geneId].transcriptDict[transcriptId].end = TSSRef
            # 修改indexTSS中相应transcript的位置
            # 在indexTSS中删除transcript的原位置
            indexTSS[chr][TSS][TES] = [i for i in indexTSS[chr][TSS][TES] if i!=transcriptId]
            if not indexTSS[chr][TSS][TES]:
                indexTSS[chr][TSS].pop(TES)
            if not indexTSS[chr][TSS]:
                indexTSS[chr].pop(TSS)
            # 在indexTSS中新增transcript的新位置
            if TSSRef not in indexTSS[chr]:
                indexTSS[chr][TSSRef] = {TES: [transcriptId]}
            else:
                if TES not in indexTSS[chr][TSSRef]:
                    indexTSS[chr][TSSRef][TES] = [transcriptId]
                else:
                    indexTSS[chr][TSSRef][TES].append(transcriptId)
            # transcriptIntegrate中添加该映射
            transcriptIntegrate["TSS"][transcriptId] = TSSRef
            # 删除transcriptSet中的这些transcript
            transcriptSet.discard(transcriptId)


In [29]:
# 合并TES
# 根据TES建立index
indexTES = total.transcriptIndexBuild(siteType="TES")
# 筛选出TES端不可信的transcript
transcriptSet = set()
for geneId, geneObject in total.geneDict.items():
    for transcriptId, transcriptObject in geneObject.transcriptDict.items():
        status = transcriptObject.status
        if status == "KNOWN" or "-TES" in status:
            # 过滤掉TES端可信的transcript
            continue
        else:
            transcriptSet.add((geneId, transcriptId))
# 只要transcriptSet不为空集, 就提取transcriptSet中的一个transcriptId
while transcriptSet:
    temp = []  # 存储该transcript与周围的transcript  [(<geneId>, <transcriptId>, <expression>), ...]
    (geneId, transcriptId) = transcriptSet.pop()
    chr = total.geneDict[geneId].chr
    transcriptObject = total.geneDict[geneId].transcriptDict[transcriptId]
    TES = transcriptObject.TES
    # 寻找其TSS上下游各RANGETES范围内的transcript(不包括TES)
    # 递减寻找RANGETES范围内的transcript
    temp = temp + findId(total=total, index=indexTES, chr=chr, location=TES, rangeMax=RANGETES, orientation='-', allowType="transcript", transcriptGeneMap=transcriptGeneMap)
    # 递增寻找RANGETES范围内的transcript
    temp = temp + findId(total=total, index=indexTES, chr=chr, location=TES, rangeMax=RANGETES, orientation='+', allowType="transcript", transcriptGeneMap=transcriptGeneMap)
    # 判断transcript附近是否存在transcriptAdjacent
    if temp:
        existedMarker = True
    else:
        existedMarker = False
        # transcript附近不存在transcriptAdjacent, 该transcript的TES无法修改, 跳过该transcript
        continue
    # temp中添加所有与该transcript的TES相同的transcriptAdjacent
    for transcriptIdList in indexTES[chr][TES].values():
        for transcriptId in transcriptIdList:
            geneId = transcriptGeneMap[transcriptId]
            temp = temp + [(geneId, transcriptId, numpy.mean([total.geneDict[geneId].transcriptDict[transcriptId].cellLineExpression.get(cellLine, 0) for cellLine in total.celllineInfo.keys()]))]
    # 去重, 虽然不应该存在重复
    temp = list(set(temp))
    # 判断在temp中是否存在已校正端
    # 分离其中的未校正端与已校正端
    corSet = set()  # {(geneId, transcriptId, TSS), ...}
    nonCorSet = set()  # {(geneId, transcriptId, TSS), ...}
    for (geneId, transcriptId, expression) in temp:
        status = total.geneDict[geneId].transcriptDict[transcriptId].status
        TES = total.geneDict[geneId].transcriptDict[transcriptId].TES
        if status=="KNOWN" or "-TES" in status:
            # TES端可信
            corSet.add((geneId, transcriptId, TES))
        else:
            # TES端不可信
            nonCorSet.add((geneId, transcriptId, TES))
    if corSet:
        # 存在已校正端
        # 对每一个未校正端选择其距离最近的已校正端作为参考位点进行校正
        for (geneId, transcriptId, TES) in nonCorSet:
            chr = total.geneDict[geneId].chr
            TSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
            # 寻找最近的已校正端
            diffTES = [(geneIdTemp, transcriptIdTemp, abs(TES-TESTemp)) for (geneIdTemp, transcriptIdTemp, TESTemp) in corSet]
            diffTES = sorted(diffTES, key=lambda x: x[2])
            (geneRefId, transcriptRefId, diffNum) = diffTES[0]
            TESRef = total.geneDict[geneRefId].transcriptDict[transcriptRefId].TES
            # 修改TES, start/end位置
            total.geneDict[geneId].transcriptDict[transcriptId].TES = TESRef
            if total.geneDict[geneId].strand == '+':
                total.geneDict[geneId].transcriptDict[transcriptId].end = TESRef
            else:
                total.geneDict[geneId].transcriptDict[transcriptId].start = TESRef
            # 修改indexTES中相应transcript的位置
            # 在indexTES中删除transcript的原位置
            indexTES[chr][TES][TSS] = [i for i in indexTES[chr][TES][TSS] if i!=transcriptId]
            if not indexTES[chr][TES][TSS]:
                indexTES[chr][TES].pop(TSS)
            if not indexTES[chr][TES]:
                indexTES[chr].pop(TES)
            # 在indexTES中新增transcript的新位置
            if TESRef not in indexTES[chr]:
                indexTES[chr][TESRef] = {TSS: [transcriptId]}
            else:
                if TSS not in indexTES[chr][TESRef]:
                    indexTES[chr][TESRef][TSS] = [transcriptId]
                else:
                    indexTES[chr][TESRef][TSS].append(transcriptId)
            # transcriptIntegrate中添加该映射
            transcriptIntegrate["TES"][transcriptId] = TESRef
            # 改变status
            total.geneDict[geneId].transcriptDict[transcriptId].status = total.geneDict[geneId].transcriptDict[transcriptId].status + "-TES"
            # 删除transcriptSet中该transcriptId
            transcriptSet.discard(transcriptId)
    else:
        # 不存在已校正端
        # 确定TESRef
        # 建立TES位点与transcript表达的关系, 选择表达量最高的TES作为TESRef
        TESRef = {}
        for (geneId, transcriptId, expression) in temp:
            TES = total.geneDict[geneId].transcriptDict[transcriptId].TES
            TESExpression = TESRef.get(TES, 0)
            TESExpression = TESExpression + expression
            TESRef[TES] = TESExpression
        TESRef = [(TES, expression) for TES, expression in TESRef.items()]
        TESRef = sorted(TESRef, key=lambda x: x[1], reverse=True)
        TESRef = TESRef[0][0]
        # 修改temp中所有transcript的TES, start/end
        for (geneId, transcriptId, expression) in temp:
            TSS = total.geneDict[geneId].transcriptDict[transcriptId].TSS
            TES = total.geneDict[geneId].transcriptDict[transcriptId].TES
            total.geneDict[geneId].transcriptDict[transcriptId].TES = TESRef
            if total.geneDict[geneId].strand == '+':
                total.geneDict[geneId].transcriptDict[transcriptId].end = TESRef
            else:
                total.geneDict[geneId].transcriptDict[transcriptId].start = TESRef
            
            # 修改indexTES中相应transcript的位置
            # 在indexTES中删除transcript的原位置
            indexTES[chr][TES][TSS] = [i for i in indexTES[chr][TES][TSS] if i!=transcriptId]
            if not indexTES[chr][TES][TSS]:
                indexTES[chr][TES].pop(TSS)
            if not indexTES[chr][TES]:
                indexTES[chr].pop(TES)
            # 在indexTES中新增transcript的新位置
            if TESRef not in indexTES[chr]:
                indexTES[chr][TESRef] = {TSS: [transcriptId]}
            else:
                if TSS not in indexTES[chr][TESRef]:
                    indexTES[chr][TESRef][TSS] = [transcriptId]
                else:
                    indexTES[chr][TESRef][TSS].append(transcriptId)
            # transcriptIntegrate中添加该映射
            transcriptIntegrate["TES"][transcriptId] = TESRef
            # 删除transcriptSet中的这些transcript
            transcriptSet.discard(transcriptId)

In [30]:
# 纠正transcript的status
for geneObject in total.geneDict.values():
    for transcriptObject in geneObject.transcriptDict.values():
        status = transcriptObject.status
        if status == "CORRECTED-TES-TSS":
            transcriptObject.status = "CORRECTED-TSS-TES"
        elif status == "NOVEL-TSS":
            transcriptObject.status = "CORRECTED-TSS"
        elif status == "NOVEL-TES":
            transcriptObject.status = "CORRECTED-TES"
        elif status == "NOVEL-TSS-TES":
            transcriptObject.status = "CORRECTED-TSS-TES"
        elif status == "NOVEL-TES-TSS":
            transcriptObject.status = "CORRECTED-TSS-TES"

In [31]:
# 重建index, 并重新计算表达量
total.reIndex()
total.computeRelativeExpression()
total.computeCellLineExpression()

In [31]:
# debug
with open("10002_corrected.pkl", "wb") as file:
    pickle.dump(total, file)

In [8]:
# debug
with open("10002_corrected.pkl", "rb") as file:
    total = pickle.load(file)

In [35]:
# 根据基因组注释校正gene
# 声明变量
geneSet = set()
geneCorrected = {"mapped": {}, "TSS-TES": {}, "TSS": {}, "TES": {}}
totalGeneIndex = total.geneIndexBuild(siteType="TSS")
# 统计校正前及gene的数量
num = 0
for geneObject in total.geneDict.values():
    status = geneObject.status
    if status == "NOVEL":
        num += 1
infoDict["novel gene before gene correct"] = num
num = 0
for geneObject in total.geneDict.values():
    num += 1
infoDict["total gene before gene correct"] = num
# 遍历所有status为KNOWN的gene, 校正其TSS及TES端
for geneObject in total.geneDict.values():
    status = geneObject.status
    if status == "KNOWN":
        geneId = geneObject.geneId
        geneObject.start = geneRef[geneId]["start"]
        geneObject.end = geneRef[geneId]["end"]
        geneObject.TSS = geneRef[geneId]["TSS"]
        geneObject.TES = geneRef[geneId]["TES"]
# 遍历所有status为NOVEL的gene, 将所有geneId存储至变量geneSet中
for geneObject in total.geneDict.values():
    status = geneObject.status
    if status == "NOVEL":
        geneSet.add(geneObject.geneId)
# 寻找gene的映射对象geneRef
while geneSet:
    geneId = geneSet.pop()
    geneObject = total.geneDict[geneId]
    geneList = []
    chr = geneObject.chr
    TSS = geneObject.TSS
    TES = geneObject.TES
    # 根据TSS获取其上下游geneRangeTSS范围内的gene
    temp = [geneIndex["TSS"][chr].get(i, None) for i in range(TSS-GENERANGETSS, TSS+GENERANGETSS+1)]
    # 去除空值
    while None in temp:
        temp.remove(None)
    while {} in temp:
        temp.remove({})
    for d in temp:
        for tempTES, tempGeneRefId in d.items():
            if abs(TES - tempTES) <= GENERANGETES:
                # 从中筛选出[TES-GENERANGETES, TES+GENERANGETES]范围的所有gene, 认为这些gene为双端可映射的gene
                num = abs(TES - tempTES) + abs(TSS - geneRef[tempGeneRefId]["TSS"])
                geneList.append((tempGeneRefId, num, True, True))
            else:
                # 仅TSS映射到了tempGeneRef
                num = abs(TSS - geneRef[tempGeneRefId]["TSS"])
                geneList.append((tempGeneRefId, num, True, False))
    # 根据TES获取其上下游geneRangeTES范围内的gene
    temp = [geneIndex["TES"][chr].get(i, None) for i in range(TES-GENERANGETES, TES+GENERANGETES+1)]
    # 去除空值
    while None in temp:
        temp.remove(None)
    while {} in temp:
        temp.remove({})
    for d in temp:
        for tempTSS, tempGeneRefId in d.items():
            # 从中筛选出(-oo, TSS-GENERANGETSS)U(TSS+GENERANGETSS, +oo)的所有gene, 认为这些gene为仅可映射到TES的gene
            if abs(tempTSS-TSS) > GENERANGETSS:
                # 仅TES可映射到tempGeneRef
                num = abs(TES-geneRef[tempGeneRefId]["TES"])
                geneList.append((tempGeneRefId, num, False, True))
    # 准备映射
    # 判断是否存在映射
    if not geneList:
        # 不存在映射, 跳过该gene
        continue
    # 判断映射的类型
    # 优先判断是否仅一个映射
    if len(geneList) == 1:
        # 仅单个映射
        (geneRefId, num, markerTSS, markerTES) = geneList[0]
        geneRefSet = {(geneRefId, num, markerTSS, markerTES, None, None)}
    else:
        # 其次筛选出双端映射的geneRef
        doubleMapped = []
        for (geneRefId, num, markerTSS, markerTES) in geneList:
            if markerTSS == markerTES == True:
                doubleMapped.append([geneRefId, num, markerTSS, markerTES])
        # 判断是否存在双端映射的geneRef
        if doubleMapped:
            # 存在双端映射的geneRef
            doubleMapped = sorted(doubleMapped, key=lambda x: x[1])
            # 筛选出相差碱基数最少的geneRef
            (geneRefId, num, markerTSS, markerTES) = doubleMapped[0]
            geneRefSet = {(geneRefId, num, markerTSS, markerTES, None, None)}
        else:
            # 不存在双端映射的geneRef
            geneRefSet = set()  # 可能有一个元素, 也可能有两个元素
            # TSS端
            geneTSSList = [(geneRefId, num, markerTSS, markerTES) for (geneRefId, num, markerTSS, markerTES) in geneList if markerTSS is True]
            if geneTSSList:
                geneTSSList = sorted(geneTSSList, key=lambda x: x[1])
                # 筛选出TSS端相差碱基数最少的geneRef
                (geneRefId, num, markerTSS, markerTES) = geneTSSList[0]
                geneRefSet = geneRefSet.union({(geneRefId, num, markerTSS, markerTES, None, None)})
            # TES端
            geneTESList = [(geneRefId, num, markerTSS, markerTES) for (geneRefId, num, markerTSS, markerTES) in geneList if markerTES is True]
            if geneTESList:
                geneTESList = sorted(geneTESList, key=lambda x: x[1])
                # 筛选出TES端相差碱基数最少的geneRef
                (geneRefId, num, markerTSS, markerTES) = geneTESList[0]
                geneRefSet = geneRefSet.union({(geneRefId, num, markerTSS, markerTES, None, None)})
    # 检查geneRef是否已记录
    temp = [[geneRefId, num, markerTSS, markerTES, marker, mapped] for (geneRefId, num, markerTSS, markerTES, marker, mapped) in geneRefSet]
    for index, value in enumerate(temp):
        geneRefId = temp[index][0]
        if geneRefId in total.geneDict:
            # geneRefId已记录
            temp[index][4] = 1
            temp[index][5] = None
        else:
            # geneRefId未记录, 检查geneRef是否可映射到其他gene
            geneRefChr = geneRef[geneRefId]["chr"]
            geneRefTSS = geneRef[geneRefId]["TSS"]
            geneRefTES = geneRef[geneRefId]["TES"]
            if geneRefTSS not in totalGeneIndex[geneRefChr]:
                # geneRef的TSS未记录, geneRef未记录于total
                temp[index][4] = 3
                temp[index][5] = None
            else:
                # geneRef的TSS已记录, 检查TES是否记录
                if geneRefTES not in totalGeneIndex[geneRefChr][geneRefTSS]:
                    # geneRef的TES未记录, geneRef未记录于total
                    temp[index][4] = 3
                    temp[index][5] = None
                else:
                    # geneRef已存在于total中
                    # 检查已存在的gene是否是KNOWN类型的gene
                    existedGeneId = totalGeneIndex[geneRefChr][geneRefTSS][geneRefTES]
                    if total.geneDict[existedGeneId].status == "KNOWN":
                        temp[index][4] = 2
                        temp[index][5] = existedGeneId
                    else:
                        temp[index][4] = 3
                        temp[index][5] = None
    geneRefSet = {(geneRefId, num, markerTSS, markerTES, marker, mapped) for [geneRefId, num, markerTSS, markerTES, marker, mapped] in temp}
    # 开始映射
    # 检查geneRefSet的长度, 若长度为2则表明gene的两端分别映射到了两个geneRef, 否则则表明gene映射到了单个geneRef
    if len(geneRefSet) == 2:
        # 分别映射到两geneRef
        geneCorrected["TSS-TES"][geneId] = {"TSS": None, "TES": None}
        oldChr = total.geneDict[geneId].chr
        oldTSS = total.geneDict[geneId].TSS
        oldTES = total.geneDict[geneId].TES
        # 遍历geneRefSet
        for (geneRefId, num, markerTSS, markerTES, marker, mapped) in geneRefSet:
            # 根据geneRef, 修改gene的TSS, start/end
            if markerTSS is True:
                total.geneDict[geneId].TSS = geneRef[geneRefId]["TSS"]
                if total.geneDict[geneId].strand == '+':
                    total.geneDict[geneId].start = geneRef[geneRefId]["start"]
                else:
                    total.geneDict[geneId].end = geneRef[geneRefId]["end"]
                geneCorrected["TSS-TES"][geneId]["TSS"] = geneRef[geneRefId]["TSS"]
            # 根据geneRef, 修改gene的TES, start/end
            else:
                total.geneDict[geneId].TES = geneRef[geneRefId]["TES"]
                if total.geneDict[geneId].strand == '+':
                    total.geneDict[geneId].end = geneRef[geneRefId]["end"]
                else:
                    total.geneDict[geneId].start = geneRef[geneRefId]["start"]
                geneCorrected["TSS-TES"][geneId]["TES"] = geneRef[geneRefId]["TES"]
        # 根据geneRef, 修改gene的status为CORRECTED-TSS-TES
        total.geneDict[geneId].status = "CORRECTED-TSS-TES"
        # 修改totalGeneIndex中该gene的位置
        totalGeneIndex[oldChr][oldTSS].pop(oldTES)
        if not totalGeneIndex[oldChr][oldTSS]:
            totalGeneIndex[oldChr].pop(oldTSS)
        newTSS = total.geneDict[geneId].TSS
        newTES = total.geneDict[geneId].TES
        if newTSS not in totalGeneIndex[oldChr]:
            totalGeneIndex[oldChr][newTSS] = {newTES: geneId}
        else:
            if newTES not in totalGeneIndex[oldChr][newTSS]:
                totalGeneIndex[oldChr][newTSS][newTES] = geneId
            else:
                pass
    else:
        # 映射到单个geneRef
        (geneRefId, num, markerTSS, markerTES, marker, mapped) = geneRefSet.pop()
        if markerTSS == markerTES == True:
            # 双端映射
            if marker == 1:
                # geneRefId已记录
                idChr = total.geneDict[geneId].chr
                idTSS = total.geneDict[geneId].TSS
                idTES = total.geneDict[geneId].TES
                # 将gene合并至geneRef
                (deletedId, remainedId) = total._geneMerge(geneId1=geneRefId, geneId2=geneId)
            elif marker == 2:
                # geneRef已记录, 但id为mapped
                geneRefId = mapped
                idChr = total.geneDict[geneId].chr
                idTSS = total.geneDict[geneId].TSS
                idTES = total.geneDict[geneId].TES
                # 将gene合并至geneRef
                (deletedId, remainedId) = total._geneMerge(geneId1=geneRefId, geneId2=geneId)
            elif marker == 3:
                # geneRef未记录
                # 添加geneRef对象
                total.geneAdd(status="KNOWN",
                              chr=geneRef[geneRefId]["chr"],
                              strand=geneRef[geneRefId]["strand"],
                              start=geneRef[geneRefId]["start"],
                              end=geneRef[geneRefId]["end"],
                              geneId=geneRefId,
                              geneVersion=geneRef[geneRefId]["geneVersion"],
                              geneName=geneRef[geneRefId]["geneName"],
                              geneBiotype=geneRef[geneRefId]["geneBiotype"])
                # 将gene合并至geneRef
                (deletedId, remainedId) = total._geneMerge(geneId1=geneRefId, geneId2=geneId)
            else:
                raise ValueError("marker should be 1 or 2 or 3, but it is {}".format(marker))
            # 修改totalGeneIndex中该gene的位置
            refChr = total.geneDict[geneRefId].chr
            refTSS = total.geneDict[geneRefId].TSS
            refTES = total.geneDict[geneRefId].TES
            if deletedId == geneId:
                (deletedChr, deletedTSS, deletedTES) = (idChr, idTSS, idTES)
                (remainedChr, remainedTSS, remainedTES) = (refChr, refTSS, refTES)
            else:
                (deletedChr, deletedTSS, deletedTES) = (refChr, refTSS, refTES)
                (remainedChr, remainedTSS, remainedTES) = (idChr, idTSS, idTES)
            # 删除totalGeneIndex中的gene
            totalGeneIndex[deletedChr][deletedTSS].pop(deletedTES)
            if not totalGeneIndex[deletedChr][deletedTSS]:
                totalGeneIndex[deletedChr].pop(deletedTSS)
            if remainedTSS not in totalGeneIndex[remainedChr]:
                totalGeneIndex[remainedChr][remainedTSS] = {remainedTES: remainedId}
            else:
                if remainedTES not in totalGeneIndex[remainedChr][remainedTSS]:
                    totalGeneIndex[remainedChr][remainedTSS][remainedTES] = remainedId
            # 记录该映射到geneCorrected["mapped"]
            geneCorrected["mapped"][deletedId] = remainedId
        else:
            # 仅单端映射
            chr = total.geneDict[geneId].chr
            oldTSS = total.geneDict[geneId].TSS
            oldTES = total.geneDict[geneId].TES
            if markerTSS is True:
                # 仅TSS端可映射
                # 根据geneRef, 修改gene的TSS, start/end
                total.geneDict[geneId].TSS = geneRef[geneRefId]["TSS"]
                newTSS = geneRef[geneRefId]["TSS"]
                if total.geneDict[geneId].strand == '+':
                    total.geneDict[geneId].start = geneRef[geneRefId]["start"]
                else:
                    total.geneDict[geneId].end = geneRef[geneRefId]["end"]
                # 修改gene的status为CORRECTED-TSS
                total.geneDict[geneId].status = "CORRECTED-TSS"
                # 修改totalGeneIndex中该gene的位置
                totalGeneIndex[chr][oldTSS].pop(oldTES)
                if not totalGeneIndex[chr][oldTSS]:
                    totalGeneIndex[chr].pop(oldTSS)
                if newTSS not in totalGeneIndex[chr]:
                    totalGeneIndex[chr][newTSS] = {oldTES: geneId}
                else:
                    if oldTES not in totalGeneIndex[chr][newTSS]:
                        totalGeneIndex[chr][newTSS][oldTES] = geneId
                # 记录该映射到geneCorrected["TSS"]
                geneCorrected["TSS"][geneId] = newTSS
            else:
                # 仅TES端可映射
                # 根据geneRef, 修改gene的TSS, start/end
                total.geneDict[geneId].TES = geneRef[geneRefId]["TES"]
                newTES = geneRef[geneRefId]["TES"]
                if total.geneDict[geneId].strand == '+':
                    total.geneDict[geneId].end = geneRef[geneRefId]["end"]
                else:
                    total.geneDict[geneId].start = geneRef[geneRefId]["start"]
                # 修改gene的status为CORRECTED-TES
                total.geneDict[geneId].status = "CORRECTED-TES"
                # 修改totalGeneIndex中该gene的位置
                totalGeneIndex[chr][oldTSS].pop(oldTES)
                totalGeneIndex[chr][oldTSS][newTES] = geneId
                # 记录该映射到geneCorrected["TES"]
                geneCorrected["TES"][geneId] = newTES
# 重建index并重新计算relativeExpression及cellLineExpression
total.reIndex()
total.computeRelativeExpression()
total.computeCellLineExpression()

In [37]:
# 合并gene
# 声明变量
geneSet = set()  # {<geneId>, ...}
geneIntegrate = {"TSS": {}, "TES": {}}  # {"TSS": {<geneId>: <TSSRef>, ...}, "TES": {<geneId>: <TESRef>, ...}}
# 根据TSS合并gene
# 根据TSS建立index
indexTSS = total.geneIndexBuild(siteType="TSS")
# 遍历所有gene, 去除KNOWN或TSS已校正的gene, 将剩余的gene添加到geneSet
geneSet = set()
for geneId, geneObject in total.geneDict.items():
    status = geneObject.status
    if status == "KNOWN" or "-TSS" in status:
        # 过滤掉TSS端可信的transcript
        continue
    else:
        geneSet.add(geneId)
# 只要geneSet不为空集, 就提取geneSet中的一个geneId
while geneSet:
    temp = []  # 存储该gene与周围的gene  [(<geneId>, <expression>), ...]
    geneId = geneSet.pop()
    geneObject = total.geneDict[geneId]
    chr = geneObject.chr
    TSS = geneObject.TSS
    # 寻找其TSS上下游各GENERANGETSS范围内的gene(不包括TSS)
    # 递减寻找GENERANGETSS范围内的gene
    temp = temp + findId(total=total, index=indexTSS, chr=chr, location=TSS, rangeMax=GENERANGETSS, orientation='-', allowType="gene")
    # 递增寻找GENERANGETSS范围内的gene
    temp = temp + findId(total=total, index=indexTSS, chr=chr, location=TSS, rangeMax=GENERANGETSS, orientation='+', allowType="gene")
    # 判断gene附近是否存在geneAdjacent
    if not temp:
        # gene附近不存在geneAdjacent, 该gene的TSS无法修改, 跳过该gene
        continue
    # 能寻找到gene, 进行下一步
    # temp中添加所有与该gene的TSS相同的geneAdjacent
    for geneId in indexTSS[chr][TSS].values():
        temp = temp + [(geneId, numpy.mean([total.geneDict[geneId].cellLineExpression.get(cellLine, 0) for cellLine in total.celllineInfo.keys()]))]
    # 去重, 虽然不应该存在重复
    temp = list(set(temp))
    print(geneId, temp)
    # 判断在temp中是否存在已校正端
    # 分离其中的未校正端与已校正端的gene
    corSet = set()  # {(geneId, TSS), ...}
    nonCorSet = set()  # {(geneId, TSS), ...}
    for (geneId, expression) in temp:
        status = total.geneDict[geneId].status
        TSS = total.geneDict[geneId].TSS
        if status=="KNOWN" or "-TSS" in status:
            # TSS端可信
            corSet.add((geneId, TSS))
        else:
            # TSS端不可信
            nonCorSet.add((geneId, TSS))
    if corSet:
        # 存在已校正端的gene
        # 对每一个未校正端选择其距离最近的已校正端作为参考位点进行校正
        for (geneId, TSS) in nonCorSet:
            chr = total.geneDict[geneId].chr
            TES = total.geneDict[geneId].TES
            # 寻找最近的已校正端
            diffTSS = [(geneIdTemp, abs(TSS-TSSTemp)) for (geneIdTemp, TSSTemp) in corSet]
            diffTSS = sorted(diffTSS, key=lambda x: x[1])
            (geneRefId, diffNum) = diffTSS[0]
            TSSRef = total.geneDict[geneRefId].TSS
            # 修改TSS, start/end位置
            total.geneDict[geneId].TSS = TSSRef
            if total.geneDict[geneId].strand == '+':
                total.geneDict[geneId].start = TSSRef
            else:
                total.geneDict[geneId].end = TSSRef
            # 修改indexTSS中相应gene的位置
            # 在indexTSS中删除gene的原位置
            indexTSS[chr][TSS].pop(TES)
            if not indexTSS[chr][TSS]:
                indexTSS[chr].pop(TSS)
            # 在indexTSS中新增gene的新位置
            if TSSRef not in indexTSS[chr]:
                indexTSS[chr][TSSRef] = {TES: geneId}
            else:
                indexTSS[chr][TSSRef][TES] = geneId
            # geneIntegrate中添加该映射
            geneIntegrate["TSS"][geneId] = TSSRef
            # 改变status
            total.geneDict[geneId].status = total.geneDict[geneId].status + "-TSS"
            # 删除geneSet中该geneId
            geneSet.discard(geneId)
    else:
        # 若不存在已校正端, 则进行下一步
        # 确定TSSRef
        # 建立TSS位点与gene表达的关系, 选择表达量最高的TSS作为TSSRef
        TSSRef = {}
        for (geneId, expression) in temp:
            TSS = total.geneDict[geneId].TSS
            TSSExpression = TSSRef.get(TSS, 0)
            TSSExpression = TSSExpression + expression
            TSSRef[TSS] = TSSExpression
        TSSRef = [(TSS, expression) for TSS, expression in TSSRef.items()]
        TSSRef = sorted(TSSRef, key=lambda x: x[1], reverse=True)
        TSSRef = TSSRef[0][0]
        # 修改temp中所有gene的TSS, start/end
        for (geneId, expression) in temp:
            TSS = total.geneDict[geneId].TSS
            TES = total.geneDict[geneId].TES
            total.geneDict[geneId].TSS = TSSRef
            if total.geneDict[geneId].strand == '+':
                total.geneDict[geneId].start = TSSRef
            else:
                total.geneDict[geneId].end = TSSRef
            # 修改indexTSS中相应gene的位置
            # 在indexTSS中删除gene的原位置
            indexTSS[chr][TSS].pop(TES)
            if not indexTSS[chr][TSS]:
                indexTSS[chr].pop(TSS)
            # 在indexTSS中新增gene的新位置
            if TSSRef not in indexTSS[chr]:
                indexTSS[chr][TSSRef] = {TES: geneId}
            else:
                indexTSS[chr][TSSRef][TES] = geneId
            # geneIntegrate中添加该映射
            geneIntegrate["TSS"][geneId] = TSSRef
            # 删除geneSet中该geneId
            geneSet.discard(geneId)
# 根据TES合并gene
# 根据TES建立index
indexTES = total.geneIndexBuild(siteType="TES")
# 遍历所有gene, 去除KNOWN或TES已校正的gene, 将剩余的gene添加到geneSet
geneSet = set()
for geneId, geneObject in total.geneDict.items():
    status = geneObject.status
    if status == "KNOWN" or "-TES" in status:
        # 过滤掉TES端可信的transcript
        continue
    else:
        geneSet.add(geneId)
# 只要geneSet不为空集, 就提取geneSet中的一个geneId
while geneSet:
    temp = []  # 存储该gene与周围的gene  [(<geneId>, <expression>), ...]
    geneId = geneSet.pop()
    geneObject = total.geneDict[geneId]
    chr = geneObject.chr
    TES = geneObject.TES
    # 寻找其TES上下游各GENERANGETES范围内的gene(不包括TES)
    # 递减寻找GENERANGETES范围内的gene
    temp = temp + findId(total=total, index=indexTES, chr=chr, location=TES, rangeMax=GENERANGETES, orientation='-', allowType="gene")
    # 递增寻找GENERANGETES范围内的gene
    temp = temp + findId(total=total, index=indexTES, chr=chr, location=TES, rangeMax=GENERANGETES, orientation='+', allowType="gene")
    # 判断gene附近是否存在geneAdjacent
    if not temp:
        # gene附近不存在geneAdjacent, 该gene的TES无法修改, 跳过该gene
        continue
    # 能寻找到gene, 进行下一步
    # temp中添加所有与该gene的TES相同的geneAdjacent
    for geneId in indexTES[chr][TES].values():
        temp = temp + [(geneId, numpy.mean([total.geneDict[geneId].cellLineExpression.get(cellLine, 0) for cellLine in total.celllineInfo.keys()]))]
    # 去重, 虽然不应该存在重复
    temp = list(set(temp))
    # 判断在temp中是否存在已校正端
    # 分离其中的未校正端与已校正端的gene
    corSet = set()  # {(geneId, TES), ...}
    nonCorSet = set()  # {(geneId, TES), ...}
    for (geneId, expression) in temp:
        status = total.geneDict[geneId].status
        TES = total.geneDict[geneId].TES
        if status=="KNOWN" or "-TES" in status:
            # TES端可信
            corSet.add((geneId, TES))
        else:
            # TES端不可信
            nonCorSet.add((geneId, TES))
    if corSet:
        # 存在已校正端的gene
        # 对每一个未校正端选择其距离最近的已校正端作为参考位点进行校正
        for (geneId, TES) in nonCorSet:
            chr = total.geneDict[geneId].chr
            TSS = total.geneDict[geneId].TSS
            # 寻找最近的已校正端
            diffTSS = [(geneIdTemp, abs(TES-TESTemp)) for (geneIdTemp, TESTemp) in corSet]
            diffTSS = sorted(diffTSS, key=lambda x: x[1])
            (geneRefId, diffNum) = diffTSS[0]
            TESRef = total.geneDict[geneRefId].TES
            # 修改TES, start/end位置
            total.geneDict[geneId].TES = TESRef
            if total.geneDict[geneId].strand == '+':
                total.geneDict[geneId].end = TESRef
            else:
                total.geneDict[geneId].start = TESRef
            # 修改indexTES中相应gene的位置
            # 在indexTES中删除gene的原位置
            indexTES[chr][TES].pop(TSS)
            if not indexTES[chr][TES]:
                indexTES[chr].pop(TES)
            # 在indexTSS中新增gene的新位置
            if TESRef not in indexTES[chr]:
                indexTES[chr][TESRef] = {TSS: geneId}
            else:
                indexTES[chr][TESRef][TSS] = geneId
            # geneIntegrate中添加该映射
            geneIntegrate["TES"][geneId] = TESRef
            # 改变status
            total.geneDict[geneId].status = total.geneDict[geneId].status + "-TES"
            # 删除geneSet中该geneId
            geneSet.discard(geneId)
    else:
        # 若不存在已校正端, 则进行下一步
        # 确定TESRef
        # 建立TES位点与gene表达的关系, 选择表达量最高的TES作为TESRef
        TESRef = {}
        for (geneId, expression) in temp:
            TES = total.geneDict[geneId].TES
            TESExpression = TESRef.get(TES, 0)
            TESExpression = TESExpression + expression
            TESRef[TES] = TESExpression
        TESRef = [(TES, expression) for TES, expression in TESRef.items()]
        TESRef = sorted(TESRef, key=lambda x: x[1], reverse=True)
        TESRef = TESRef[0][0]
        # 修改temp中所有gene的TES, start/end
        for (geneId, expression) in temp:
            TSS = total.geneDict[geneId].TSS
            TES = total.geneDict[geneId].TES
            total.geneDict[geneId].TES = TESRef
            if total.geneDict[geneId].strand == '+':
                total.geneDict[geneId].end = TESRef
            else:
                total.geneDict[geneId].start = TESRef
            # 修改indexTES中相应gene的位置
            # 在indexTES中删除gene的原位置
            indexTES[chr][TES].pop(TSS)
            if not indexTES[chr][TES]:
                indexTES[chr].pop(TES)
            # 在indexTES中新增gene的新位置
            if TESRef not in indexTES[chr]:
                indexTES[chr][TESRef] = {TSS: geneId}
            else:
                indexTES[chr][TESRef][TSS] = geneId
            # geneIntegrate中添加该映射
            geneIntegrate["TES"][geneId] = TESRef
            # 删除geneSet中该geneId
            geneSet.discard(geneId)

ENCLB767XEKG000089686 [('ENCLB767XEKG000089686', 0.41535483763779396), ('ENCLB315YZPG000093925', 0.33057239270942423), ('ENCLB240AAXG000088174', 0.39816810816794096)]
ENCLB240AAXG000078190 [('ENCLB345FISG000081779', 0.48434437530876956), ('ENCLB240AAXG000078190', 0.5972521622519114), ('ENCLB315YZPG000081744', 2.1487205526112576)]
ENCLB767XEKG000067420 [('ENCLB767XEKG000067420', 4.3612257951968365), ('ENCLB315YZPG000068610', 2.975151534384818), ('ENCLB345FISG000068291', 3.9391224512535166)]
ENCLB315YZPG000075091 [('ENCLB315YZPG000075091', 0.9917171781282726), ('ENCLB240AAXG000072475', 0.5972521622519114)]
ENCLB240AAXG000062404 [('ENCLB345FISG000063288', 0.6457925004116927), ('ENCLB240AAXG000062404', 0.7963362163358819)]
ENCLB345FISG000088951 [('ENCLB767XEKG000085438', 1.0383870940944848), ('ENCLB315YZPG000089095', 0.33057239270942423), ('ENCLB345FISG000088951', 0.6457925004116927)]
ENCLB240AAXG000085445 [('ENCLB240AAXG000085445', 0.9954202704198524), ('ENCLB315YZPG000090627', 1.32228957

KeyError: 25548317

In [32]:
num = {}
for geneObject in total.geneDict.values():
    status = geneObject.status
    num.setdefault(status, 0)
    num[status] += 1
num

{'KNOWN': 17488, 'NOVEL': 58318}

---

start

In [ ]:
# 简要思路
'''
exon:
    校正方法: 
        1. 对于每个exon, 寻找基因组注释中该exon的start与end上下游各5bp范围内的所有exon, 共有四种映射类型
            1. 双端单映射, 该exon的start及end端都可映射到基因组注释中的同一个exon
            2. 双端双映射, 该exon的start及end端可分别映射到基因组注释中的exon
            3. 单端单映射, 该exon仅start端可映射到基因组注释中的exon
            4. 单端单映射, 该exon仅end端可映射到基因组注释中的exon
        2. 筛选参考exon
            优先级: 双端单映射 > 双端双映射 > 单端单映射
            1. 在这些参考exon的集合中, 优先筛选出双端单映射的exon, 并进一步筛选出相差碱基数最少的exon作为最终的参考exon
            2. 其次才筛选双端双映射的exon, 并筛选出相差碱基数最少的exon作为最终的参考exon
            3. 最后才筛选单端单映射的exon作为参考exon
        3. 校正exon
            根据参考exon的信息对当前exon的start及end进行修改
    合并方法:
        1. 对于已记录的exon, 分别筛选出start, end未经过校正的exon
        2. 以合并start端为例
        3. 寻找其中每个exon的start上下游各5bp范围内的所有exon, 当在该范围内寻找到exon时再以该exon为中心继续寻找其附近的exon, 最终形成一个集合, 这个集合中所有exon的start上下游5bp范围内不再有其他exon
        4. 若这个集合中存在已校正的start端, 则将所有未校正的exon的start修改为距离其最近的已校正的start
        5. 若这个集合中不存在已校正的start端, 则对每个exon取其在所有细胞系中的平均表达值, 选择表达最高的exon的start作为集合中所有exon的start
'''

In [ ]:
# 具体操作
'''
根据参考基因组进行校正的思路
0. 处理exon
    0.0 声明变量
        exonCorrected, dict, {"mapped": {<novel exon id>: <known exon id>, ...},
                              "TSS": {<novel exon id>: <known exon id>, ...},
                              "TES": {<novel exon id>: <known exon id>, ...},
                              "TSS-TES": {<novel exon id>: <known exon id>, ...}}
        readyCorList, list, 存储status为NOVEL的exonId[<exonId>, ...]
    0.1 引用变量
        EXONRANGE, int, exon的start及end允许的偏差范围
    已弃置 0.2 根据end建立索引
    0.3 根据参考基因组中exon数据对exon的TSS及TES进行校正
        因为三代测序中出现的错误绝大部分属于碱基的插入/缺失, 所以需要对exon的TSS/TES进行校正
        读取参考基因组注释中exon的数据, 并建立index
        统计校正前exon及transcript的数量
        遍历total.exonDict中所有exon对象, 判断每个exon上下游exonRange范围内是否存在参考exon
        声明list类型变量exonList, 存储该exon所能映射的所有参考exon, [[<exonRef>, <bool>, <bool>], ...]
        准备校正exon的TSS及TES
            获取[TSS-5, TES+5]范围的所有exon
                从中筛选出[TES-5, TES+5]范围的所有exon, 认为这些exon为可双端映射的exon
                未筛选出的exon则认为是仅可映射到TSS的exon
            获取[TES-5, TES+5]范围的所有exon
                从中筛选出(-oo, TSS-5)U(TSS+5, +oo)的所有exon, 认为这些exon为仅可映射到TES的exon
            在获取的所有exon中
                优先筛选出可双端映射的exon, 并在这些exon中选择总体距离最近的exon进行映射
                其次才筛选出仅可单端映射的exon, 并在这些exon中选择离单端距离最近的exon进行映射
        校正过程
            若不存在映射, 则跳过该exon, 分析下一个exon
            若存在双端映射exon
                获取相差碱基数最少的exon作为参考exon
                判断exonId是否相同
                若exonId相同, 则修改status, exonVersion, start, TSS, end, TES
                若exonId不同, 则查询参考exonId是否已记录
                    若参考exonId已记录, 则合并两exon
                    若参考exonId未记录
                        exonDict中新增参考exon对象
                        合并原exon与参考exon
                    将这种映射关系添加到exonCorrected["mapped"]中
            若仅存在单端映射exon
                如果status为KNOWN就不再进行映射, 跳过该exon
                修改exon的status为CORRECTED
                判断exon能否双端映射到两个exon
                    能映射到两个exon
                        在exon的status中添加-TSS-TES后缀
                        获取TSS端相差碱基数最少的exon
                        修改exon的start,end,TSS的位置
                        获取TES端相差碱基数最少的exon
                        修改exon的start,end,TES的位置
                        将映射关系{<exonId>: <newExonId>}保存到exonCorrected["TSS-TES"]中
                    不能映射到两个exon, 则进行下一步
                筛选出相差碱基数最少的exon进行映射
                若仅TSS端可映射
                    在exon的status中添加-TSS后缀
                    修改exon的start,end,TSS的位置
                    将映射关系{<exonId>: <newExonId>}保存到exonCorrected["TSS"]中
                若仅TES端可映射
                    在exon的status中添加-TES后缀
                    修改exon的start,end,TES的位置
                    将映射关系{<exonId>: <newExonId>}保存到exonCorrected["TES"]中
    已弃置 因为已根据参考基因组进行了校正
        0.3 处理total中的exon数据
        遍历total.exonDict中所有.status=="NOVEL"的exon, 记录这些exon
        声明list类型变量exonList, 存储该exon所能映射的所有参考exon, [[<exonRef>, <bool>, <bool>], ...]
        准备校正exon的start及end
            获取[start-5, start+5]范围的所有exon
                从中筛选出[end-5, end+5]范围的所有exon, 认为这些exon为可双端映射的exon
                未筛选出的exon则认为是仅可映射到start的exon
            获取[end-5, end+5]范围的所有exon
                从中筛选出(-oo, start-5)U(start+5, +oo)的所有exon, 认为这些exon为仅可映射到end的exon
            在获取的所有exon中
                优先筛选出可双端映射的exon, 并在这些exon中选择总体距离最近的exon进行映射
                其次才筛选出仅可单端映射的exon, 并在这些exon中选择离单端距离最近的exon进行映射
        校正过程
            若exon可双端映射到一个exon
                获取相差碱基数最少的exon
                在total.exonExisted中添加该映射
                删除total.exonDict中该exon对象
                将映射关系{<exonId>: <newExonId>}保存到exonCorrected["mapped"]中
            若exon不可双端映射到一个exon
                修改exon的status为CORRECTED
                判断exon能否双端映射到两个exon
                    能映射到两个exon
                        在exon的status中添加-start-end后缀
                        获取start端相差碱基数最少的exon
                        修改exon的start的位置
                        获取end端相差碱基数最少的exon
                        修改exon的end的位置
                        将映射关系{<exonId>: <newExonId>}保存到exonCorrected["start-end"]中
                    不能映射到两个exon, 则进行下一步
                筛选出相差碱基数最少的exon进行映射
                若仅start端可映射
                    在exon的status中添加-start后缀
                    修改exon的start的位置
                    将映射关系{<exonId>: <newExonId>}保存到exonCorrected["start"]中
                若仅end端可映射
                    在exon的status中添加-end后缀
                    修改exon的end的位置
                    将映射关系{<exonId>: <newExonId>}保存到exonCorrected["end"]中
    0.4 更新total中的exonDict及exonIndex
        更新transcriptDict以合并可能重复的transcript(因为exon组成可能重复)
        重建Index, 合并重复的exon及transcript
    0.5 合并相邻的exon, 即合并相差EXONRANGE范围内的NOVEL的exon
        情况说明
            在合并距离较近的exon时可能会出现这么一种情况
            当EXONRANGE=5时, 200可以合并到205, 205又可以合并到210, 所以200就可以合并到210
            因此需要对这种情况进行处理, 而不是简单的将两exon根据EXONRANGE进行合并
            所以需要先形成一个簇, 然后对这个簇中所有的exon的TSS或TES进行校正
        准备记录合并记录 
        建立index
            声明indexTSS, {<chr>: {<TSS>: {<TES>: <exonId>, ...}, ...}, ...}
            声明indexTES, {<chr>: {<TES>: {<TSS>: <exonId>, ...}, ...}, ...}
            遍历total.exonDict
                过滤掉所有status==KNOWN的exon
                若status中不含-TSS, 则在indexTSS中添加该exon信息
                若status中不含-TES, 则在indexTES中添加该exon信息
                否则, continue
        遍历total.exonDict中所有status不为KNOWN及其中一端未校正的exon
            处理TSS
                声明list变量exonSet, {<exonId>, ...}
                遍历所有exon, 去除KNOWN或TSS已校正的exon, 将剩余的exon添加到exonSet
                只要exonSet不为空集, 就随机提取exonSet中的一个exonId
                    temp中添加所有与该exon的TSS相同的exon
                    寻找其TSS上下游各EXONRANGE范围内的exon
                        若寻找不到其他的exon, 就认为该exon的TSS无法修改, 跳过该exon
                        若能寻找到其他的exon, 就进行下一步
                判断在temp中是否存在已校正端
                    分离其中的未校正端与已校正端
                    若存在已校正端
                        对每一个未校正端选择其距离最近的已校正端作为参考位点进行校正
                            寻找最近的已校正端
                            修改TSS, start/end位置
                            改变status
                            删除exonSet中该exonId
                        结束此次循环
                    若不存在已校正端, 则进行下一步
                以expression最大的exon为参照, 修改其他exon的TSS
                    寻找expression最大的exon
                    修改其他exon的TSS, start/end
                    修改status
                    删除exonSet中的这些exon
                删除exonSet中的这些exon
                示例
                    权重: 相对表达量
                        假设有7个exon, 从start=100开始寻找EXONRANGE=5范围的exonStart
                        exon0   95  200
                        exon1   99  201
                        exon2   99  200
                        exon3   100 200
                        exon4   104 204
                        exon5   106 206
                        exon6   107 210
                        exon7   110 210
                    分别递增, 递减寻找EXONRANGE范围内的exon
                        当递减寻找[100-5, 100)的start时, 寻找到了99
                            temp中添加[exon1, expression]
                            temp中添加[exon2, expression]
                            此时再根据99去递减寻找[99-5, 99)范围的start, 寻找到了95
                                temp中添加[exon0, expression]
                            此时再根据95去递减寻找[95-5, 95)范围的start, 寻找到了None
                                当寻找到None时, 该寻找过程终止
                        当递增寻找(100, 100+5]的start时, 寻找到了104
                            temp中添加[exon4, expression]
                            此时再根据104去递增寻找(104, 104+5]范围的start, 寻找到了106
                                temp中添加[exon5, expression]
                            此时再根据106去递增寻找(106, 106+5]范围的start, 寻找到了107
                                temp中添加[exon6, expression]
                            ...
                            此时再根据***去递增寻找(***, ***+5]范围的start, 寻找到了None
                                当寻找到None时, 寻找过程终止
                    当两个寻找过程都停止后, 则开始对temp中的exon的start进行校正
                        筛选temp中expression最大的exon, 作为参考exon
                        其他exon的start都将修改为参考exon的start
            处理TES
                声明list变量temp, [[<end>, <expression>], ...]
                权重: 相对表达量
                分别递增, 递减寻找EXONRANGE范围内的exon, 在temp中添加[exon, expression]
                当两个寻找过程都停止后, 则开始对temp中的exon的end进行校正
                    与start处理过程相似
    0.6 在所有exon都处理完后
        更新transcriptDict以合并可能重复的transcript(因为exon组成可能重复)
        重建Index
1. 校正transcript
    1.0 声明变量
        transcriptSet, set, 用于存储transcriptId {<transcriptId>, ...}
        transcriptCorrected, set, 用于存储校正过程中transcript的变化情况
        transcriptGeneMap, dict, 存储transcriptId与geneId的映射关系
        totalTranscriptIndex, dict, 准备total中transcript的index
    1.1 读取参考基因组注释数据
        获取所有transcript信息
        补充transcript中的exon信息
        获取index, {"TSS": {<chr>: {<TSS>: {<TES>: [<transcriptId>, ...],
                                                         ...},
                                                 ...},
                                         ...},
                                 "TES": {<chr>: {<TES>: {<TSS>: [<transcriptId>, ...],
                                                         ...},
                                                 ...},
                                         ...}
                                 }
    1.2 统计校正前及transcript的数量
    1.3 遍历所有status为KNOWN的transcript, 校正其TSS及TES端
        根据transcriptId, 在transcriptRef中检索, 修改其TSS/TES, start/end
        此时, 可认为status为KNOWN的transcript的start/end, TSS/TES是准确的
    1.4 遍历所有status为NOVEL的transcript, 将所有transcriptId存储至变量transcriptSet中
        认为这些transcript的start/end/TSS/TES都是不可靠的, 需要进行校正
    1.5 寻找transcriptSet中可映射的transcript
        声明变量transcriptList, 存储该transcript所能映射的所有参考transcript, [[<transcriptRefId>, <num>, <bool>, <bool>], ...]
        在基因组注释的index中, 根据TSS获取其上下游rangeTSS范围内的transcript
            在这些transcript中, 筛选TES上下游在rangeTES范围内的transcript, 认为这些transcript是可双端映射的transcript
            剩余的transcript则被认为是仅可校正TSS端的transcript
        在基因组注释的index中, 根据TES获取其上下游rangeTES范围内的transcript
            在这些transcript中, 筛选TSS上下游不在rangeTSS范围内的transcript, 认为这些transcript是仅可校正TES端的transcript
    1.6 准备映射
        判断是否存在映射, 若不存在映射则跳过该transcript, 若存在映射则进行下一步
        判断映射的类型
            判断是否仅一个映射
                若仅单个映射就直接存储为transcriptRefSet={(<transcriptRefId>, <num>, <bool>, <bool>, <marker>, <mapped id>)}
            优先筛选出双端映射的transcript
                (已弃置, 不应该存在id相同的项)最优先筛选出transcriptId相同的transcript作为transcriptRef
                其次筛选出exon组成相似程度最高的transcript作为transcriptRef
                    # exon组成相似程度如此设定的原因: 在exon数量差异与单个exon位点差异中, 更倾向于选择exon数量相似但每个exon不同的transcript
                    # 因为存在较多的非KNOWN的exon, 这些exon其实可以归类到一个KNOWN的exon, 只是因为测序过程中出现插入/删除以及校正参数的问题这些exon没有归类到一个exon中
                    exon组成相似程度 = (transcriptRef的exon数量 与 transcript的exon数量 中最小的数/最大的数) * transcript与transcriptRef重叠的exon数量
                    若exon组成相似程度相同, 则筛选出TSS与TES相差碱基数最少的transcript作为transcriptRef
                transcriptRef存储为transcriptRefSet={(<transcriptRefId>, <num>, True, True, <marker>, <mapped id>)}
            无双端映射transcript, 仅有单端映射transcript
                transcriptRefSet=set()  # 可能有一个元素, 也可能有两个元素
                TSS端
                    (已弃置)最优先筛选出transcriptId相同的transcript作为transcriptRef
                    其次筛选出exon组成最相似的transcript作为transcriptRef
                        exon组成相似程度 = (transcriptRef的exon数量 与 transcript的exon数量 中最小的数/最大的数) * transcript与transcriptRef重叠的exon数量
                        若exon组成相似程度相同, 则筛选出TSS与TES相差碱基数最少的transcript作为transcriptRef
                    transcriptRefSet中添加(<transcriptRefId>, True, False, <marker>, <mapped id>)
                TES端
                    (已弃置)最优先筛选出transcriptId相同的transcript作为transcriptRef
                    其次筛选出exon组成最相似的transcript作为transcriptRef
                        exon组成相似程度 = (transcriptRef的exon数量 与 transcript的exon数量 中最小的数/最大的数) * transcript与transcriptRef重叠的exon数量
                        若exon组成相似程度相同, 则筛选出TSS与TES相差碱基数最少的transcript作为transcriptRef
                    transcriptRefSet中添加(<transcriptRefId>, False, True, <marker>, <mapped id>)
        遍历transcriptRefSet, 检查transcriptRef是否已记录
            检查transcriptRefId是否已记录
                若参考transcriptId已记录, 认为该transcript已记录
                    marker = 1
                    mapped id = None
                若transcriptRefId未记录, 则进行下一步
            检查geneObject.transcriptIndex中是否已记录该transcript
                若已记录
                    判断已记录的那个transcript是否为NOVEL类型的transcript
                    若transcriptRef映射的那个transcript属于NOVEL类型transcript
                        marker = 3
                        mapped id = None
                    若那个transcript属于KNOWN类型的transcript
                        marker = 2
                        mapped id = transcriptRefId
                若未记录, 进行下一步
                    marker = 3
                    mapped id = None
    1.7 开始映射
        检查transcriptRefSet的长度为2则分别映射到两个transcriptRef, 否则为映射到单个transcriptRef
        若长度为2, 分别映射到两transcriptRef
            遍历transcriptRefSet
                根据transcriptRef, 修改transcript的TSS, start/end
                根据transcriptRef, 修改transcript的TES, start/end
                根据transcriptRef, 修改transcript的status为CORRECTED-TSS-TES
                记录该映射到transcriptCorrected["TSS-TES"]
        若长度为1, 映射到单个transcript
            若双端映射
                判断transcript与transcriptRef的exon组成是否一致
                    若exon组成组成一致, 则可以合并
                    若exon组成不一致, 则不可合并
                若可合并
                    检查transcriptRef的记录状态marker
                        若transcriptRef已记录
                            将transcript合并至transcriptRef
                            记录该映射到transcriptCorrected["mapped"]
                        若transcriptRef未记录
                            检查transcriptRef所属的geneRef是否已记录
                                检查geneRefId是否已记录
                                    若已记录则认为geneRef已记录
                                    若未记录则检查geneRefId是否映射到了别的geneRefId
                                        若已映射, 则认为geneRef已记录
                                        若未映射, 则添加该geneRef对象
                            在对应的geneRefObject中添加transcriptRef对象
                            合并transcript至transcriptRef
                    记录这一次合并到transcriptCorrected["mapped"]
                若不可合并
                    根据transcriptRef修改transcript的start, end, TSS, TES
                    修改transcript的status为"CORRECTED-TSS-TES"
                    记录该映射到transcriptCorrected["TSS-TES"]
            若单端映射
                若仅TSS端可映射
                    根据transcriptRef, 修改transcript的TSS, start/end
                    根据transcriptRef, 修改transcript的status为CORRECTED-TSS
                    记录该映射到transcriptCorrected["TSS"]
                若仅TES端可映射
                    根据transcriptRef, 修改transcript的TES, start/end
                    根据transcriptRef, 修改transcript的status为CORRECTED-TES
                    记录该映射到transcriptCorrected["TES"]
    1.8 重建index, 重新计算relativeExpression及cellLineExpression
2. 合并transcript
    2.0 声明变量
        transcriptSet, set, 用于存储TSS/TES可进行合并的transcript
        transcriptIntegrate, dict, 用于存储TSS/TES合并的信息
        transcriptGeneMap, dict, 存储total中transcript与所属gene的对应关系  {<transcriptId>: <geneId>}
    2.1 对TSS进行合并
        根据TSS建立index
        遍历所有transcript, 去除KNOWN或TSS已校正的transcript, 将剩余的transcript添加到transcriptSet
        只要transcriptSet不为空集, 就提取transcriptSet中的一个transcriptId
            寻找其TSS上下游各RANGETSS范围内的transcript(不包括TSS)
                若寻找不到transcript, 就认为该transcript的TSS无法修改, 跳过该transcript
                若能寻找到transcript, 就进行下一步
            temp中添加所有与该transcript的TSS相同的transcriptAdjacent
        判断在temp中是否存在含有已校正端的transcript
            分离其中的未校正端与已校正端的transcript
            若存在已校正端的transcript
                对每一个未校正端transcript选择其距离最近的已校正端作为参考位点进行校正
                修改TSS, start/end位置
                修改indexTSS中相应transcript的位置
                    在indexTSS中删除transcript的原位置
                    在indexTSS中新增transcript的新位置
                transcriptIntegrate中添加该映射
                改变status
                删除transcriptSet中该transcriptId
            若不存在已校正端, 则进行下一步
                确定TSSRef
                建立TSS位点与transcript表达的关系, 选择表达量最高的TSS作为参考TSS
                修改temp中所有transcript的TSS, start/end
                修改indexTSS中相应transcript的位置
                    在indexTSS中删除transcript的原位置
                    在indexTSS中新增transcript的新位置
                transcriptIntegrate中添加该映射
                删除transcriptSet中该transcriptId
    2.2 对TES进行合并
        同2.1的方法
    2.3 在所有transcript都处理完后
        更新transcriptDict以合并可能重复的transcript
        重建Index
        重新computeRelativeExpression()
        重新computeCellLineExpression()
3. 根据基因组注释校正gene
    3.0 声明变量
        geneSet, set, 用于存储geneId {<geneId>, ...}
        geneCorrected, set, 用于存储校正过程中gene的变化情况
        totalGeneIndex, dict, 准备total中gene的index
    3.1 统计校正前及gene的数量
    3.2 遍历所有status为KNOWN的gene, 校正其TSS及TES端
        根据geneId, 在geneRef中检索, 修改其TSS/TES, start/end
        此时, 可认为status为KNOWN的gene的start/end, TSS/TES是准确的
    3.3 遍历所有status为NOVEL的gene, 将所有geneId存储至变量geneSet中
        认为这些gene的start/end/TSS/TES都是不可靠的, 需要进行校正
    3.4 寻找geneSet中可映射的gene
        声明变量geneList, 存储该gene所能映射的所有geneRef, [[<geneRefId>, <num>, <bool>, <bool>], ...]
        在基因组注释的index中, 根据TSS获取其上下游geneRangeTSS范围内的geneRef
            在这些geneRef中, 筛选TES上下游在geneRangeTES范围内的geneRef, 认为这些geneRef是可双端映射的geneRef
            剩余的geneRef则被认为是仅可校正TSS端的geneRef
        在基因组注释的index中, 根据TES获取其上下游geneRangeTES范围内的geneRef
            在这些geneRef中, 筛选TSS上下游不在geneRangeTSS范围内的geneRef, 认为这些geneRef是仅可校正TES端的geneRef
    3.5 准备映射
        判断是否存在映射, 若不存在映射则跳过该gene, 若存在映射则进行下一步
            判断映射的类型
                优先判断是否仅一个映射
                    若仅单个映射就直接存储为geneRefSet={(<geneRefId>, <num>, <bool>, <bool>, <marker>, <mapped id>)}
                其次筛选出双端映射的geneRef
                    筛选出与gene相差碱基数最少的geneRef作为待映射geneRef
                    geneRef存储为geneRefSet={(<geneRefId>, <num>, True, True, <marker>, <mapped id>)}
                最后才筛选单端映射geneRef
                    geneRefSet=set()  # 可能有一个元素, 也可能有两个元素
                    TSS端
                        筛选出与gene相差碱基数最少的geneRef作为待映射geneRef
                        geneRefSet中添加{(<geneRefId>, <num>, <bool>, <bool>, <marker>, <mapped id>)}
                    TES端
                        筛选出与gene相差碱基数最少的geneRef作为待映射geneRef
                        geneRefSet中添加{(<geneRefId>, <num>, <bool>, <bool>, <marker>, <mapped id>)}
        遍历geneRefSet, 检查其中记录的geneRef是否已记录
            检查geneRefId是否已记录
                若geneRefId已记录, 则认为该gene已记录
                    marker = 1
                    mapped id = None
                若geneRefId未记录, 则判断该geneRefId是否已映射到其他id
                    检查totalGeneIndex中是否已记录该gene
                        若已记录, 则判断已记录的那个gene是否为NOVEL类型的gene
                            若geneRef映射到的那个gene属于NOVEL类型的gene
                                marker = 3
                                mapped id = None
                            若geneRef映射到的那个gene属于KNOWN类型的gene
                                marker = 2
                                mapped id = geneRefId
                        若未记录
                            marker = 3
                            mapped id = None
    3.6 开始映射
        检查geneRefSet的长度, 若长度为2则表明gene的两端分别映射到了两个geneRef, 否则则表明gene映射到了单个geneRef
        若geneRefSet的长度为2
            遍历geneRefSet
                根据geneRef, 修改gene的TSS, start/end
                根据geneRef, 修改gene的TES, start/end
                根据geneRef, 修改gene的status为CORRECTED-TSS-TES
                修改totalGeneIndex中该gene的位置
                记录该映射到geneCorrected["TSS-TES"]
        若geneRefSet的长度为1
            若为双端单映射
                检查geneRef的记录状态marker
                    若geneRef已记录
                        将gene合并至geneRef
                    若geneRef未记录
                        添加geneRef对象
                        将gene合并至geneRef
                修改totalGeneIndex中该gene的位置
                记录该映射到geneCorrected["mapped"]
            若仅TSS端可映射
                根据geneRef, 修改gene的TSS, start/end
                根据geneRef, 修改gene的status为CORRECTED-TSS
                修改totalGeneIndex中该gene的位置
                记录该映射到geneCorrected["TSS"]
            若仅TES端可映射
                根据geneRef, 修改gene的TES, start/end
                根据geneRef, 修改gene的status为CORRECTED-TES
                修改totalGeneIndex中该gene的位置
                记录该映射到geneCorrected["TES"]
    3.7 重建index, 重新计算relativeExpression及cellLineExpression
4. 合并gene
    4.0 声明变量
        geneSet, set, 用于存储TSS/TES可进行合并的gene
        geneIntegrate, dict, 用于存储TSS/TES合并的信息
    4.1 根据TSS合并gene
        根据TSS建立index
        遍历所有gene, 去除KNOWN或TSS已校正的gene, 将剩余的gene添加到geneSet
        只要geneSet不为空集, 就提取geneSet中的一个geneId
            寻找其TSS上下游各GENERANGETSS范围内的gene(不包括TSS)
                若寻找不到gene, 就认为该gene的TSS无法修改, 跳过该gene
                若能寻找到gene, 就进行下一步
            temp中添加所有与该gene的TSS相同的geneAdjacent
        判断在temp中是否存在含有已校正端的gene
            分离其中的未校正端与已校正端的gene
            若存在已校正端的gene
                对每一个未校正端gene选择其距离最近的已校正端作为参考位点进行校正
                修改TSS, start/end位置
                修改indexTSS中相应gene的位置
                    在indexTSS中删除gene的原位置
                    在indexTSS中新增gene的新位置
                geneIntegrate中添加该映射
                改变status
                删除geneSet中该geneId
            若不存在已校正端, 则进行下一步
                确定TSSRef
                    建立TSS位点与gene表达的关系, 选择表达量最高的TSS作为TSSRef
                修改temp中所有gene的TSS, start/end
                修改indexTSS中相应gene的位置
                    在indexTSS中删除gene的原位置
                    在indexTSS中新增gene的新位置
                geneIntegrate中添加该映射
                删除geneSet中该geneId
    4.2 根据TES合并gene
        同4.1的方法
    4.3 在所有gene都处理完后
        纠正gene的status
        重建Index
        重新computeRelativeExpression()
        重新computeCellLineExpression()
'''

In [37]:
# exon校正策略2
exonSet = set()
exonCorrected = {"mapped": {}, "TSS-TES": {}, "TSS": {}, "TES": {}}
totalExonIndex = total.exonIndexBuild(siteType="TSS")

In [ ]:
'''# 测试: 根据参考基因组进行校正
for exonObject in total.exonDict.values():
    status = exonObject.status
    if status == "KNOWN":
        exonId = exonObject.exonId
        exonObject.start = exonRef[exonId]["start"]
        exonObject.end = exonRef[exonId]["end"]
        exonObject.TSS = exonRef[exonId]["TSS"]
        exonObject.TES = exonRef[exonId]["TES"]
# 遍历所有status为NOVEL的exon, 将所有exonId存储至exonSet中
for exonObject in total.exonDict.values():
    status = exonObject.status
    if status == "NOVEL":
        exonSet.add(exonObject.exonId)
# 寻找exon的映射对象exonRef
while exonSet:
    exonId = exonSet.pop()
    exonObject = total.exonDict[exonId]
    exonList = []
    chr = exonObject.chr
    TSS = exonObject.TSS
    TES = exonObject.TES
    # 根据TSS获取其上下游EXONRANGE范围内的exon
    temp = [exonIndex["TSS"][chr].get(i, None) for i in range(TSS-EXONRANGE, TSS+EXONRANGE+1)]
    # 去除空值
    while None in temp:
        temp.remove(None)
    while {} in temp:
        temp.remove({})
    for d in temp:
        for tempTES, tempExonRefId in d.items():
            if abs(TES - tempTES) <= EXONRANGE:
                # 从中筛选出[TES-EXONRANGE, TES+EXONRANGE]范围的所有exon, 认为这些exon为双端可映射的exon
                num = abs(TES - tempTES) + abs(TSS - exonRef[tempExonRefId]["TSS"])
                exonList.append((tempExonRefId, num, True, True))
            else:
                # 仅TSS映射到了tempExonRef
                num = abs(TSS - exonRef[tempExonRefId]["TSS"])
                exonList.append((tempExonRefId, num, True, False))
    # 根据TES获取其上下游EXONRANGE范围内的exon
    temp = [exonIndex["TES"][chr].get(i, None) for i in range(TES-EXONRANGE, TES+EXONRANGE+1)]
    # 去除空值
    while None in temp:
        temp.remove(None)
    while {} in temp:
        temp.remove({})
    for d in temp:
        for tempTSS, tempExonRefId in d.items():
            # 从中筛选出(-oo, TSS-EXONRANGE)U(TSS+EXONRANGE, +oo)的所有exon, 认为这些exon为仅可映射到TES的exon
            if abs(tempTSS-TSS) > EXONRANGE:
                # 仅TES可映射到tempExonRef
                num = abs(TES-exonRef[tempExonRefId]["TES"])
                exonList.append((tempExonRefId, num, False, True))
    # 准备映射
    # 判断是否存在映射
    if not exonList:
        # 不存在映射, 跳过该exon
        continue
    # 判断映射的类型
    # 优先判断是否仅一个映射
    if len(exonList) == 1:
        # 仅单个映射
        (exonRefId, num, markerTSS, markerTES) = exonList[0]
        exonRefSet = {(exonRefId, num, markerTSS, markerTES, None, None)}
    else:
        # 其次筛选出双端映射的exonRef
        doubleMapped = []
        for (exonRefId, num, markerTSS, markerTES) in exonList:
            if markerTSS == markerTES == True:
                doubleMapped.append([exonRefId, num, markerTSS, markerTES])
        # 判断是否存在双端映射的exonRef
        if doubleMapped:
            # 存在双端映射的exonRef
            doubleMapped = sorted(doubleMapped, key=lambda x: x[1])
            # 筛选出相差碱基数最少的exonRef
            (exonRefId, num, markerTSS, markerTES) = doubleMapped[0]
            exonRefSet = {(exonRefId, num, markerTSS, markerTES, None, None)}
        else:
            # 不存在双端映射的exonRef
            exonRefSet = set()  # 可能有一个元素, 也可能有两个元素
            # TSS端
            exonTSSList = [(exonRefId, num, markerTSS, markerTES) for (exonRefId, num, markerTSS, markerTES) in exonList if markerTSS is True]
            if exonTSSList:
                geneTSSList = sorted(geneTSSList, key=lambda x: x[1])
                # 筛选出TSS端相差碱基数最少的geneRef
                (geneRefId, num, markerTSS, markerTES) = geneTSSList[0]
                geneRefSet = geneRefSet.union({(geneRefId, num, markerTSS, markerTES, None, None)})
            # TES端
            geneTESList = [(geneRefId, num, markerTSS, markerTES) for (geneRefId, num, markerTSS, markerTES) in geneList if markerTES is True]
            if geneTESList:
                geneTESList = sorted(geneTESList, key=lambda x: x[1])
                # 筛选出TES端相差碱基数最少的geneRef
                (geneRefId, num, markerTSS, markerTES) = geneTESList[0]
                geneRefSet = geneRefSet.union({(geneRefId, num, markerTSS, markerTES, None, None)})
    # 检查geneRef是否已记录'''

In [26]:
num = {}
for geneObject in total.geneDict.values():
    status = geneObject.status
    num.setdefault(status, 0)
    num[status] += 1
print(num)

{'KNOWN': 17488, 'NOVEL': 57304, 'CORRECTED-TSS': 491, 'CORRECTED-TES': 474, 'CORRECTED-TSS-TES': 29}


---

暂存部分

end

---

---

In [2]:
import plotly.graph_objects as go
trace = go.Pie(labels=["novel transcript", "start-end corrected", "start corrected", "end corrected", "known transcript"],
	   values=[69998, 57452, 15695, 37945, 27433],
	   marker={"colors": ["#505168", "#2A7F62", "#75DDDD", "#E8E1EF", "lightgrey"],
			   "line": {"color": "white",
						"width": 1}},
	   sort=False,
	   textinfo="value",
	   textfont_size=12,
	   insidetextorientation='auto',
	   hole=0,
	   pull=[0.1, 0, 0, 0, 0],
	   )
fig = go.Figure(trace)

layout={"width": 500, "height": 500,
		"margin": {'l':25, 'r':25, 't':25, 'b':25},
        "title": {"text": "Correction of transcript<br>TSS: 50  TES: 150",
				  "font": {"family": "Arial",
						   "color": "black",
						   "size": 16},
				   "x": 0.4},
        "font": {"family": "Arial",  # 作用所有地方
				 "color": "black",
				 "size": 12,
				 },
        "paper_bgcolor": "white",
        "plot_bgcolor": "white",
        }
fig = fig.update_layout(layout)
'''
将最大校正范围设定在5bp时, 在标注为novel的74504个exon中, 有3558个exon可以完全映射到已注释exon上, 有16229个exon的start附近存在已注释exon, 有10967个exon的end附近存在已注释exon。
已根据其映射情况修改了exon的start或end的位置
'''

'\n将最大校正范围设定在5bp时, 在标注为novel的74504个exon中, 有3558个exon可以完全映射到已注释exon上, 有16229个exon的start附近存在已注释exon, 有10967个exon的end附近存在已注释exon。\n已根据其映射情况修改了exon的start或end的位置\n'

In [3]:
fig.show()
fig.write_image("transcript.png")

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 创建子图，两个 Y 轴
fig = make_subplots(specs=[[{"secondary_y": True}]])

trace1 = go.Scatter(x=[1,2,3,4,5,6,7,8,9,10,
                       11,12,13,14,15,16,17,18,19,20],
                   y=[213,934,1826,3197,3558,3867,4071,4233,4416,4559,
                      4677,4825,4914,5005,5138,5225,5308,5437,5546,5630],
                   mode="lines+markers",
                   name="start-end exon",
                   line={"color": "#2A7F62"})
trace2 = go.Scatter(x=[1,2,3,4,5,6,7,8,9,10,
                       11,12,13,14,15,16,17,18,19,20],
                   y=[44356,44197,44197,44197,43750,43665,43603,43549,43549,43476,
                      43445,43410,43410,43346,43327,43298,43278,43257,43232,43206],
                   mode="lines+markers",
                   name="novel exon",
                   line={"color": "#505168"})

# 添加第一个 Y 轴的数据
fig.add_trace(trace1, secondary_y=False)
# 添加第二个 Y 轴的数据
fig.add_trace(trace2, secondary_y=True)

# 设置图形布局
layout = {"width": 700, "height": 400,
		    "margin": {'l':25, 'r':25, 't':25, 'b':25},
          "title": {"text": "",
				  "font": {"family": "Arial",
						   "color": "black",
						   "size": 16},
				   "x": 0.4},
        "font": {"family": "Arial",  # 作用所有地方
				 "color": "black",
				 "size": 12,
				 },
         "xaxis": {"showline": True,
                   "linecolor": "black",
                   "linewidth": 1,
                   "showticklabels": True,
                   "dtick": 2},
         "yaxis": {"showline": True,
                   "linecolor": "black",
                   "linewidth": 1,
                   "showticklabels": True,},
         "yaxis2": {"showline": True,
                   "linecolor": "black",
                   "linewidth": 1,
                   "showticklabels": True,},
        "paper_bgcolor": "white",
        "plot_bgcolor": "white",}
fig = fig.update_layout(layout)
fig = fig.update_layout(title_text="Correction of exons in different ranges",
                  xaxis_title="Range",
                  yaxis_title="Counts of start-end exon",
                  yaxis2_title="Counts of novel exon")

# 显示图形
#fig.write_image("line.png")
fig.show()
'''
之所以选择exon的最大校正范围为5bp, 是因为已经分析了在最大校正范围为1~20时的校正情况。
当范围设定在5bp时, novel exon的减少速度趋于稳定, 且在所有exon中双端可映射到已注释exon的数量的增加速度趋于稳定。
'''

不根据参考基因组注释进行校正:

校正前total exon num: 323363
校正前novel exon num: 133311
校正前total transcript num: 213387
校正前novel transcript num: 185981
mapped 6095
start-end 6503
start 22997
end 19860
校正后total exon num: 309350
校正后novel exon num: 72014
校正后total transcript num: 211068
校正后novel transcript num: 183662

根据参考基因组注释进行校正:



In [ ]:
'''	novel exon	start-end exon	start exon	end exon	remained exon
1	74504	213	17626	12309	44356
2	74504	934	17311	12062	44197
3	74504	1826	16935	11713	44197
4	74504	3197	16355	11107	44197
5	74504	3558	16229	10967	43750
6	74504	3867	16119	10853	43665
7	74504	4071	16029	10801	43603
8	74504	4233	15961	10761	43549
9	74504	4416	15877	10704	43549
10	74504	4559	15802	10667	43476
11	74504	4677	15753	10629	43445
12	74504	4825	15694	10575	43410
13	74504	4914	15651	10557	43410
14	74504	5005	15621	10532	43346
15	74504	5138	15543	10496	43327
16	74504	5225	15501	10480	43298
17	74504	5308	15452	10466	43278
18	74504	5437	15395	10415	43257
19	74504	5546	15337	10389	43232
20	74504	5630	15297	10371	43206'''
